# Data Processing

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "torch"
import torch
import keras
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from keras.layers import TorchModuleWrapper
from keras.optimizers import Adam
import json
import pandas as pd
import pprint
from json import loads, dumps
import itertools
import random
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
!git clone https://github.com/THU-KEG/MAVEN-ERE.git
!wget -O MAVEN_ERE.zip https://cloud.tsinghua.edu.cn/f/a7d1db6c44ea458bb6f0/?dl=1
!unzip MAVEN_ERE.zip
file_path = "/content/MAVEN_ERE/train.jsonl"
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file]

df = pd.DataFrame(data)



Cloning into 'MAVEN-ERE'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 177 (delta 10), reused 4 (delta 4), pack-reused 162 (from 1)
Receiving objects: 100% (177/177), 94.03 KiB | 1.52 MiB/s, done.
Resolving deltas: 100% (94/94), done.
--2024-12-08 15:48:10--  https://cloud.tsinghua.edu.cn/f/a7d1db6c44ea458bb6f0/?dl=1
Resolving cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)... 101.6.15.69, 2402:f000:1:402:101:6:15:69
Connecting to cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)|101.6.15.69|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cloud.tsinghua.edu.cn/seafhttp/files/05bd72b7-beea-4381-ae27-69507d7f1a0d/MAVEN_ERE.zip [following]
--2024-12-08 15:48:11--  https://cloud.tsinghua.edu.cn/seafhttp/files/05bd72b7-beea-4381-ae27-69507d7f1a0d/MAVEN_ERE.zip
Reusing existing connection to cloud.tsinghua.edu.cn:443.
HTTP request sent, awaiting response.

In [398]:
test_file_path = "/content/MAVEN_ERE/test.jsonl"
with open(test_file_path, 'r') as file:
    test_data = [json.loads(line) for line in file]


In [399]:
test_df = pd.DataFrame(test_data)

In [401]:
test_file_path = "/content/MAVEN_ERE/valid.jsonl"
with open(test_file_path, 'r') as file:
    valid_data = [json.loads(line) for line in file]


In [402]:
valid_df = pd.DataFrame(valid_data)

In [404]:
len(test_df)

857

In [405]:
len(df)

2913

In [ ]:
df['subevent_relations'][10]

[['EVENT_32ccc934a5ef189f43046b4a88154236',
  'EVENT_b35e6738fa7488795f0ad3aa52aaf5de'],
 ['EVENT_32ccc934a5ef189f43046b4a88154236',
  'EVENT_b8cb62622fffa1f47a756eb8a9342d6b'],
 ['EVENT_32ccc934a5ef189f43046b4a88154236',
  'EVENT_210e2093b02368a70701d24447eddbb2'],
 ['EVENT_32ccc934a5ef189f43046b4a88154236',
  'EVENT_bd5090dd9fab363da1f587295f48a47b']]

In [ ]:
df.apply(lambda x: len(x['subevent_relations']),axis=1).value_counts(ascending=True)

,count
48,1
129,1
64,1
36,1
31,1
42,1
34,1
43,1
50,1
39,1


In [ ]:
events_data = []
for doc in data:
    doc_id = doc['id']
    for event in doc.get('events', []):
          events_data.append({
              'doc_id': doc_id,
              'event_id': event['id'],
              'event_type': event['type'],
              'event_type_id': event['type_id'],
              'mention_tw': event['mention'][0]['trigger_word'],
              'sent_id': event['mention'][0]['sent_id'],
              'offset': event['mention'][0]['offset'],
            })

events_df = pd.DataFrame(events_data)
events_df

,doc_id,event_id,event_type,event_type_id,mention_tw,sent_id,offset
0,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_e2cbbfb4a5ab65856ccc7057443120e0,Control,41,ruled,1,"[38, 39]"
1,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_0b0c5d1268cc135278489b7f0004c0c3,Achieve,127,achieve,6,"[19, 20]"
2,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_e71684bf9f6e7b50bf5316982a640c15,Creating,60,creation,3,"[27, 28]"
3,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_f535eb21f8f958a04cbc69c1d1521186,Self_motion,46,Expedition,0,"[1, 2]"
4,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_d2587256a67ce75ea9d9d864288d822c,Motion,10,landed,1,"[21, 22]"
...,...,...,...,...,...,...,...
67979,eb724b2d3f906f8dd7aabc218adba5dd,EVENT_dffddcbaa25cc39d65c87b6f5caf2c06,Cause_change_of_position_on_a_scale,37,declined,11,"[2, 3]"
67980,eb724b2d3f906f8dd7aabc218adba5dd,EVENT_7fab91a84c71e47b6d1a7dd0e2a9116e,Motion,10,drove,6,"[19, 20]"
67981,eb724b2d3f906f8dd7aabc218adba5dd,EVENT_2e811b38d1aab386a2e752d2c2b983a6,Building,44,build,6,"[38, 39]"
67982,eb724b2d3f906f8dd7aabc218adba5dd,EVENT_ddd69c03a2f138791516dc8926f412b1,Change_of_leadership,96,revolt,1,"[28, 29]"


In [421]:
events_test_data = []
for doc in valid_data:
    doc_id = doc['id']
    for event in doc.get('events', []):
          events_test_data.append({
              'doc_id': doc_id,
              'event_id': event['id'],
              'event_type': event['type'],
              'event_type_id': event['type_id'],
              'mention_tw': event['mention'][0]['trigger_word'],
              'sent_id': event['mention'][0]['sent_id'],
              'offset': event['mention'][0]['offset']
            })

events_test_df = pd.DataFrame(events_test_data)

In [ ]:
causal_data = []
for doc in data:
    doc_id = doc['id']
    for relation_type, relations in doc.get('causal_relations', {}).items():
        for relation in relations:
            causal_data.append({
                'doc_id': doc_id,
                'relation_type': relation_type,
                'cause_event': relation[0],  # The first event ID in the relation
                'effect_event': relation[1]  # The second event ID in the relation
            })

causal_df = pd.DataFrame(causal_data)
causal_dft = causal_df.reset_index()
causal_dft['pair_num'] = causal_dft.groupby('doc_id').cumcount()
causal_dft.set_index(['doc_id', 'pair_num'],drop=True,inplace=True)

causal_dft.drop('index',axis=1,inplace=True)

causal_dft

relation_type  \
doc_id                           pair_num                 
364ed14fc610df6e25a2f446e2b2d2ab 0         PRECONDITION   
                                 1         PRECONDITION   
                                 2         PRECONDITION   
                                 3         PRECONDITION   
                                 4         PRECONDITION   
...                                                 ...   
eb724b2d3f906f8dd7aabc218adba5dd 0         PRECONDITION   
                                 1         PRECONDITION   
                                 2         PRECONDITION   
                                 3         PRECONDITION   
                                 4         PRECONDITION   

                                                                      cause_event  \
doc_id                           pair_num                                           
364ed14fc610df6e25a2f446e2b2d2ab 0         EVENT_040b46602889e455df7b4327137034e9   
                                 1         EVENT_f535eb21f8f958a04cbc69c1d1521186   
                                 2         EVENT_040b46602889e455df7b4327137034e9   
                                 3         EVENT_040b46602889e455df7b4327137034e9   
                                 4         EVENT_f535eb21f8f958a04cbc69c1d1521186   
...                                                                           ...   
eb724b2d3f906f8dd7aabc218adba5dd 0         EVENT_7f0177683d2e7ae323347c5f00534a36   
                                 1         EVENT_7f0177683d2e7ae323347c5f00534a36   
                                 2         EVENT_7fab91a84c71e47b6d1a7dd0e2a9116e   
                                 3         EVENT_a498cfc917fa127a2e996d0dcc61766a   
                                 4         EVENT_7ea71127d70dcc93769b9d79291c8c61   

                                                                     effect_event  
doc_id                           pair_num                                          
364ed14fc610df6e25a2f446e2b2d2ab 0         EVENT_c027e659d7fe424a0a57ecbe35b3a7f9  
                                 1         EVENT_5445faf0caf78e3dcc1dc88e3554fac8  
                                 2         EVENT_0b0c5d1268cc135278489b7f0004c0c3  
                                 3         EVENT_931d31a625729392d27925e697762d5f  
                                 4         EVENT_0b0c5d1268cc135278489b7f0004c0c3  
...                                                                           ...  
eb724b2d3f906f8dd7aabc218adba5dd 0         EVENT_2e811b38d1aab386a2e752d2c2b983a6  
                                 1         EVENT_7fab91a84c71e47b6d1a7dd0e2a9116e  
                                 2         EVENT_2e811b38d1aab386a2e752d2c2b983a6  
                                 3         EVENT_c901467686ab9b339f843fce73eed534  
                                 4         EVENT_4809dd58af999fe79ee1a9252da7dcd5  

[36316 rows x 3 columns]

In [413]:
causal_test_data = []
for doc in valid_data:
    doc_id = doc['id']
    for relation_type, relations in doc.get('causal_relations', {}).items():
        for relation in relations:
            causal_test_data.append({
                'doc_id': doc_id,
                'relation_type': relation_type,
                'cause_event': relation[0],  # The first event ID in the relation
                'effect_event': relation[1]  # The second event ID in the relation
            })

causal_test_df = pd.DataFrame(causal_test_data)
causal_test_dft = causal_test_df.reset_index()
causal_test_dft['pair_num'] = causal_test_dft.groupby('doc_id').cumcount()
causal_test_dft.set_index(['doc_id', 'pair_num'],drop=True,inplace=True)

causal_test_dft.drop('index',axis=1,inplace=True)

causal_test_dft

relation_type  \
doc_id                           pair_num                 
c0c67db40cd5e2e03645ff1116fafcfc 0         PRECONDITION   
f28bce270df5a122c09365002d247e76 0         PRECONDITION   
                                 1         PRECONDITION   
3dcfd60153822a6a8f6a516f161fc506 0                CAUSE   
                                 1                CAUSE   
...                                                 ...   
a08c96a4152cb85e81c5c4cc2dcef166 7         PRECONDITION   
                                 8         PRECONDITION   
                                 9         PRECONDITION   
                                 10        PRECONDITION   
54d9150423d3d3c0b77bb3c929d4f5e3 0         PRECONDITION   

                                                                      cause_event  \
doc_id                           pair_num                                           
c0c67db40cd5e2e03645ff1116fafcfc 0         EVENT_c511a5c88a6b1babc55e83ec5331a6b5   
f28bce270df5a122c09365002d247e76 0         EVENT_0341c5cced5545ffe7c543b7a155bfa8   
                                 1         EVENT_e198df1a8b96d83dba53488d8d69c4e9   
3dcfd60153822a6a8f6a516f161fc506 0         EVENT_340bbca3737a4b9ca872d8cd3fe3ceff   
                                 1         EVENT_9dcd89986fe0c0efeb0c8b2f053c4587   
...                                                                           ...   
a08c96a4152cb85e81c5c4cc2dcef166 7         EVENT_998c6359d151e9d67ce9bdfb7075d59f   
                                 8         EVENT_bd3f2ab355f254aaac4069a8c507e319   
                                 9         EVENT_6d43daf8b6bc5bd245e862ec61859f60   
                                 10        EVENT_998c6359d151e9d67ce9bdfb7075d59f   
54d9150423d3d3c0b77bb3c929d4f5e3 0         EVENT_3d31558746f97cd98d946117a9d607b0   

                                                                     effect_event  
doc_id                           pair_num                                          
c0c67db40cd5e2e03645ff1116fafcfc 0         EVENT_f1e4e17fa7c45cac758d8fdbb44acd3c  
f28bce270df5a122c09365002d247e76 0         EVENT_a0a4ccf2a46f9ba0cdf1b890bfc8c968  
                                 1         EVENT_ae3d20caf515d19bfdbd12edf64a9fa7  
3dcfd60153822a6a8f6a516f161fc506 0         EVENT_9f53fb2551b9a3ac8ccce17138a95620  
                                 1         EVENT_0f9d0c0b482aab0e795f4fc522fe596d  
...                                                                           ...  
a08c96a4152cb85e81c5c4cc2dcef166 7         EVENT_35cb4d6df90dc31c2dff69fa08101cfe  
                                 8         EVENT_0696d284bb613048885b980f2ae1c9cf  
                                 9         EVENT_0696d284bb613048885b980f2ae1c9cf  
                                 10        EVENT_d94c7127be75fdb80cab1494f9df513c  
54d9150423d3d3c0b77bb3c929d4f5e3 0         EVENT_c62570a9abfb09886e1834ed5b3982b4  

[9698 rows x 3 columns]

In [ ]:
events_df['ce_type'] = 0  # Initialize with default value
events_df.loc[events_df['event_id'].isin(causal_dft['cause_event']), 'ce_type'] = 1
events_df.loc[events_df['event_id'].isin(causal_dft['effect_event']), 'ce_type'] = 2


In [449]:
events_test_df['ce_type']=0
events_test_df.loc[events_test_df['event_id'].isin(causal_test_dft['cause_event']), 'ce_type'] = 1
events_test_df.loc[events_test_df['event_id'].isin(causal_test_dft['effect_event']), 'ce_type'] = 2

In [ ]:
events_df['ce_type'].value_counts()

,count
ce_type,
0,34945
2,22599
1,10440


In [ ]:
words_df=df.rename({'id':'doc_id'},axis=1)[['doc_id','title','sentences','tokens']]
full_df=pd.merge(events_df.set_index('doc_id'), words_df.set_index('doc_id'), left_index=True, right_index=True)


In [450]:
words_test_df=valid_df.rename({'id':'doc_id'},axis=1)[['doc_id','title','sentences','tokens']]
full_test_df=pd.merge(events_test_df.set_index('doc_id'), words_test_df.set_index('doc_id'), left_index=True, right_index=True)


In [428]:
full_test_df

,event_id,event_type,event_type_id,mention_tw,sent_id,offset,title,sentences,tokens
doc_id,,,,,,,,,


In [ ]:
len(full_df)

67984

- extract list events in word order

In [ ]:
full_df['offset_start']= full_df['offset'].apply(lambda x: x[0])
full_df['offset_end']= full_df['offset'].apply(lambda x: x[1])
full_df.sort_values(by=['doc_id','sent_id','offset_start'])

,event_id,event_type,event_type_id,mention_tw,sent_id,offset,ce_type,title,sentences,tokens,offset_start,offset_end
doc_id,,,,,,,,,,,,
00061936e209b6493b73001a83a73411,EVENT_2d5d23b35a2ac9869c6d47d6e830fc17,Hostile_encounter,19,Battle,0,"[1, 2]",0,Battle of Chełm,"[The Battle of Chełm was fought on June 8, 179...","[[The, Battle, of, Chełm, was, fought, on, Jun...",1,2
00061936e209b6493b73001a83a73411,EVENT_5b57c10db35ea556497257381700e761,Hostile_encounter,19,fought,0,"[5, 6]",0,Battle of Chełm,"[The Battle of Chełm was fought on June 8, 179...","[[The, Battle, of, Chełm, was, fought, on, Jun...",5,6
00061936e209b6493b73001a83a73411,EVENT_0afb00e97d25fb0833051d2ea8411d0a,Conquering,21,took,3,"[1, 2]",0,Battle of Chełm,"[The Battle of Chełm was fought on June 8, 179...","[[The, Battle, of, Chełm, was, fought, on, Jun...",1,2
00061936e209b6493b73001a83a73411,EVENT_7fc2a0646ea4e325e65a03e32dd2ede6,Process_start,31,started,4,"[7, 8]",0,Battle of Chełm,"[The Battle of Chełm was fought on June 8, 179...","[[The, Battle, of, Chełm, was, fought, on, Jun...",7,8
00061936e209b6493b73001a83a73411,EVENT_4f5ef67f2be339676dc0c802d1bed852,Attack,23,attack,4,"[9, 10]",0,Battle of Chełm,"[The Battle of Chełm was fought on June 8, 179...","[[The, Battle, of, Chełm, was, fought, on, Jun...",9,10
...,...,...,...,...,...,...,...,...,...,...,...,...
fffddf8a2c1d0cfd603ad399e79c6836,EVENT_d78e6e6a5fc482ecfcb40fa6a353c266,Arriving,6,entering,4,"[1, 2]",1,Bombardment of Madras,[The Bombardment of Madras was an engagement o...,"[[The, Bombardment, of, Madras, was, an, engag...",1,2
fffddf8a2c1d0cfd603ad399e79c6836,EVENT_913fa8a14b2016d77da5e3bb4b32982b,Use_firearm,71,fired,4,"[24, 25]",2,Bombardment of Madras,[The Bombardment of Madras was an engagement o...,"[[The, Bombardment, of, Madras, was, an, engag...",24,25
fffddf8a2c1d0cfd603ad399e79c6836,EVENT_a18fc43f5322b295f2f24c97104809ae,Attack,23,hit,5,"[10, 11]",1,Bombardment of Madras,[The Bombardment of Madras was an engagement o...,"[[The, Bombardment, of, Madras, was, an, engag...",10,11


In [451]:
full_test_df['offset_start']= full_test_df['offset'].apply(lambda x: x[0])
full_test_df['offset_end']= full_test_df['offset'].apply(lambda x: x[1])
full_test_df.sort_values(by=['doc_id','sent_id','offset_start'])

,event_id,event_type,event_type_id,mention_tw,sent_id,offset,ce_type,title,sentences,tokens,offset_start,offset_end
doc_id,,,,,,,,,,,,
002cd40be39bbc07cc165afeeff2fd81,EVENT_36237cf9c77013be8dedcb0c4486e737,Competition,84,final,0,"[4, 5]",1,1874 FA Cup Final,[The 1874 FA Cup final was a football match be...,"[[The, 1874, FA, Cup, final, was, a, football,...",4,5
002cd40be39bbc07cc165afeeff2fd81,EVENT_38a8f9ad3422c4aa9fc7e11ab844badc,Competition,84,final,1,"[4, 5]",0,1874 FA Cup Final,[The 1874 FA Cup final was a football match be...,"[[The, 1874, FA, Cup, final, was, a, football,...",4,5
002cd40be39bbc07cc165afeeff2fd81,EVENT_38ad419564e5485aacdb62a804b4856f,Competition,84,competition,1,"[11, 12]",1,1874 FA Cup Final,[The 1874 FA Cup final was a football match be...,"[[The, 1874, FA, Cup, final, was, a, football,...",11,12
002cd40be39bbc07cc165afeeff2fd81,EVENT_fed8818c573a73267b0f1aeea7364176,Arriving,6,reached,2,"[4, 5]",0,1874 FA Cup Final,[The 1874 FA Cup final was a football match be...,"[[The, 1874, FA, Cup, final, was, a, football,...",4,5
002cd40be39bbc07cc165afeeff2fd81,EVENT_9ddfa22c7b847af2178b2cb37f9a4da3,Process_end,61,final,2,"[6, 7]",1,1874 FA Cup Final,[The 1874 FA Cup final was a football match be...,"[[The, 1874, FA, Cup, final, was, a, football,...",6,7
...,...,...,...,...,...,...,...,...,...,...,...,...
feeabdcf070b286bd5cec604092589d7,EVENT_c858ae733622e2e927effb82805f8382,Recovering,27,restore,4,"[17, 18]",2,2012 Homs offensive,[The 2012 Homs offensive was a Syrian Army off...,"[[The, 2012, Homs, offensive, was, a, Syrian, ...",17,18
feeabdcf070b286bd5cec604092589d7,EVENT_a3672a82c3e27e2c2646916fb7e81f38,Sending,7,sent,5,"[9, 10]",0,2012 Homs offensive,[The 2012 Homs offensive was a Syrian Army off...,"[[The, 2012, Homs, offensive, was, a, Syrian, ...",9,10
feeabdcf070b286bd5cec604092589d7,EVENT_e0ff37176a95ecd8d72ee7054e84183b,Having_or_lacking_access,165,cut off,6,"[12, 14]",2,2012 Homs offensive,[The 2012 Homs offensive was a Syrian Army off...,"[[The, 2012, Homs, offensive, was, a, Syrian, ...",12,14


In [ ]:
event_streams = full_df.sort_values(by=['doc_id','sent_id','offset_start']).groupby('doc_id')['mention_tw'].apply(list)
event_streams_labels = full_df.sort_values(by=['doc_id','sent_id','offset_start']).groupby('doc_id')['ce_type'].apply(list)
event_ids = full_df.sort_values(by=['doc_id','sent_id','offset_start']).groupby('doc_id')['event_id'].apply(list)

In [452]:
event_test_streams = full_test_df.sort_values(by=['doc_id','sent_id','offset_start']).groupby('doc_id')['mention_tw'].apply(list)
event_test_streams_labels = full_test_df.sort_values(by=['doc_id','sent_id','offset_start']).groupby('doc_id')['ce_type'].apply(list)
event_test_ids = full_test_df.sort_values(by=['doc_id','sent_id','offset_start']).groupby('doc_id')['event_id'].apply(list)


In [ ]:
ev_df = pd.DataFrame(event_streams).join(pd.DataFrame(event_streams_labels))

In [453]:
ev_test_df = pd.DataFrame(event_test_streams).join(pd.DataFrame(event_test_streams_labels))

In [ ]:
# Create a column identifying duplicates

events_df['off_tup'] = events_df['offset'].apply(tuple)
dup=events_df.duplicated(subset=['doc_id','sent_id', 'off_tup'])

#making sure there isnt something thats both a cause and effect
events_df[~dup]

,doc_id,event_id,event_type,event_type_id,mention_tw,sent_id,offset,ce_type,off_tup
0,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_e2cbbfb4a5ab65856ccc7057443120e0,Control,41,ruled,1,"[38, 39]",0,"(38, 39)"
1,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_0b0c5d1268cc135278489b7f0004c0c3,Achieve,127,achieve,6,"[19, 20]",2,"(19, 20)"
2,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_e71684bf9f6e7b50bf5316982a640c15,Creating,60,creation,3,"[27, 28]",0,"(27, 28)"
3,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_f535eb21f8f958a04cbc69c1d1521186,Self_motion,46,Expedition,0,"[1, 2]",2,"(1, 2)"
4,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_d2587256a67ce75ea9d9d864288d822c,Motion,10,landed,1,"[21, 22]",2,"(21, 22)"
...,...,...,...,...,...,...,...,...,...
67979,eb724b2d3f906f8dd7aabc218adba5dd,EVENT_dffddcbaa25cc39d65c87b6f5caf2c06,Cause_change_of_position_on_a_scale,37,declined,11,"[2, 3]",0,"(2, 3)"
67980,eb724b2d3f906f8dd7aabc218adba5dd,EVENT_7fab91a84c71e47b6d1a7dd0e2a9116e,Motion,10,drove,6,"[19, 20]",2,"(19, 20)"
67981,eb724b2d3f906f8dd7aabc218adba5dd,EVENT_2e811b38d1aab386a2e752d2c2b983a6,Building,44,build,6,"[38, 39]",2,"(38, 39)"
67982,eb724b2d3f906f8dd7aabc218adba5dd,EVENT_ddd69c03a2f138791516dc8926f412b1,Change_of_leadership,96,revolt,1,"[28, 29]",0,"(28, 29)"


In [454]:
events_test_df['off_tup'] = events_test_df['offset'].apply(tuple)
test_dup=events_test_df.duplicated(subset=['doc_id','sent_id', 'off_tup'])
events_test_df[test_dup]

,doc_id,event_id,event_type,event_type_id,mention_tw,sent_id,offset,ce_type,off_tup


In [ ]:
ev_df=ev_df.join(pd.DataFrame(event_ids))

In [455]:
ev_test_df=ev_test_df.join(pd.DataFrame(event_test_ids))

In [ ]:
ev_df.rename({'mention_tw': 'event_stream',
              'ce_type': 'event_stream_labels',
              'event_id': 'event_stream_ids'},
             axis=1, inplace=True)

In [456]:
ev_test_df.rename({'mention_tw': 'event_stream',
              'ce_type': 'event_stream_labels',
              'event_id': 'event_stream_ids'},
             axis=1, inplace=True)

In [ ]:
ev_df = words_df.set_index('doc_id').join(ev_df)

In [457]:
ev_test_df = words_test_df.set_index('doc_id').join(ev_test_df)

In [ ]:
ev_df.sample(5)

,title,sentences,tokens,event_stream,event_stream_labels,event_stream_ids
doc_id,,,,,,
1014db8c0922efbde41c0b0f3b0e8567,Tropical Depression Wilma (2013),"[Tropical Depression Wilma, also referred to a...","[[Tropical, Depression, Wilma, ,, also, referr...","[referred, cyclone, traveled, Forming, passed,...","[0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 1, 2, 2, 0, ...","[EVENT_27521af6386d2af5d576a736217077f1, EVENT..."
076a72afcd040c68b9979899e7788c31,2012 Summer Olympics closing ceremony,[The closing ceremony of the London 2012 Summe...,"[[The, closing, ceremony, of, the, London, 201...","[ceremony, also known as, Symphony, held, crea...","[2, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 2, 2, 2, ...","[EVENT_aba11c7986c372c48396e51ccc24a0d9, EVENT..."
6b05d9c7b0db28d0b73ae5d210e4de1b,Battle of Racibórz,[The Battle of Raciborz took place in the Poli...,"[[The, Battle, of, Raciborz, took, place, in, ...","[Battle, took place, invasion, ended, commande...","[0, 0, 0, 1, 0, 0, 0, 1, 2, 2, 1, 2, 2, 2, 0, 0]","[EVENT_75ab06973ffe504e1cc53f3427cf7297, EVENT..."
2ffb6542ea559a86c20d624700a43e36,Burma Campaign 1944,[The fighting in the Burma Campaign in 1944 wa...,"[[The, fighting, in, the, Burma, Campaign, in,...","[fighting, took place, involved, drawn, overco...","[0, 1, 0, 0, 2, 1, 1, 2, 2, 2, 0, 0, 2, 2, 2, ...","[EVENT_11f2f328e431549d795e0c013b72e434, EVENT..."
2a4a4d7d36c5b5a41a15b3c0e2e0c4f2,Sarò libera,[Sarò libera is the second solo studio album b...,"[[Sarò, libera, is, the, second, solo, studio,...","[published, preceded, released, reached, came,...","[1, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2]","[EVENT_20c2ef836536e9d0ebae671a9e765a46, EVENT..."


In [ ]:
from transformers import AutoTokenizer


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased', use_fast=True)

In [ ]:
ev_df.iloc[0]['event_stream_ids'][0]

'EVENT_f535eb21f8f958a04cbc69c1d1521186'

In [ ]:
ev_df

,title,sentences,tokens,event_stream,event_stream_labels,event_stream_ids
doc_id,,,,,,
364ed14fc610df6e25a2f446e2b2d2ab,Expedition of the Thousand,"[The Expedition of the Thousand (Italian ""Sped...","[[The, Expedition, of, the, Thousand, (, Itali...","[Expedition, took place, sailed, landed, conqu...","[2, 0, 2, 2, 2, 0, 0, 0, 1, 2, 0, 2, 1, 0, 0, ...","[EVENT_f535eb21f8f958a04cbc69c1d1521186, EVENT..."
0371bbf116422f8e3a0a853bdd1962aa,Murder of Leigh Leigh,"[The murder of Leigh Leigh, born Leigh Rennea ...","[[The, murder, of, Leigh, Leigh, ,, born, Leig...","[murder, occurred, attending, assaulted, calle...","[1, 1, 0, 0, 0, 1, 2, 2, 0, 0, 1, 2, 2, 1, 0, ...","[EVENT_1e1f24501a9f61b0e2ace8eb26c58f97, EVENT..."
097b86ef2a4c5d21037a4c6b47b3a165,Hurricane Jerry (1989),[Hurricane Jerry caused minor damage in Texas ...,"[[Hurricane, Jerry, caused, minor, damage, in,...","[caused, damage, flooding, named, developed, m...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, ...","[EVENT_a95183aadc0236e47ab433d455527c3a, EVENT..."
ab70a3e49966caa8c35f8b27fabea3ad,Death of Christopher Alder,[Christopher Alder was a trainee computer prog...,"[[Christopher, Alder, was, a, trainee, compute...","[served, commended, died, became, cause, victi...","[1, 2, 0, 0, 0, 2, 1, 2, 0, 0, 2, 1, 2, 0, 0, ...","[EVENT_3168092bd1e51a61558f9ad86ce0e097, EVENT..."
49e16a75dfd748513d5db9f8a849d218,Clackamas Town Center shooting,"[On December 11, 2012, a shooting occurred at ...","[[On, December, 11, ,, 2012, ,, a, shooting, o...","[shooting, occurred, ran, wearing, fire, fired...","[0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 2, 1, 0, 0, ...","[EVENT_ce4bef9f5ba9605323ca2c33ccf4c81e, EVENT..."
...,...,...,...,...,...,...
e16541f32886235f5c2cb9c885924c6b,1979 Football League Cup Final,[The 1979 Football League Cup Final took place...,"[[The, 1979, Football, League, Cup, Final, too...","[Final, final, played, contested, win, finishe...","[0, 0, 0, 1, 0, 2, 0]","[EVENT_38799c347ace089fb1b52f0254384597, EVENT..."
2d3d9700b6cdf5e17234280654d30439,2014 Berlin Marathon,[The 2014 Berlin Marathon was the 41st edition...,"[[The, 2014, Berlin, Marathon, was, the, 41st,...","[Marathon, took place, race, breaking, means, ...","[0, 0, 0, 1, 2, 0, 2, 1, 0, 0, 1, 2, 1, 2]","[EVENT_1d206a592e47b6b6c67443bee4615a4a, EVENT..."
ebf325c49d8dc864dd8a9df25a121812,2014 Winter Olympics opening ceremony,[The opening ceremony of the 2014 Winter Olymp...,"[[The, opening, ceremony, of, the, 2014, Winte...","[opening, ceremony, Olympics, took place, bega...","[0, 1, 0, 1, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0]","[EVENT_0fb28a8d13c7be2ac0e08baa56d6b2ad, EVENT..."


In [ ]:
ev_df=ev_df.join(df.set_index('id')['causal_relations'])
causal_df= causal_df.set_index('cause_event').join(events_df.set_index('event_id').loc[causal_dft['cause_event']]['mention_tw']).rename({'mention_tw':'cause_mention_tw'},axis=1).reset_index().set_index('effect_event').join(events_df.set_index('event_id').loc[causal_dft['effect_event']]['mention_tw']).rename({'mention_tw':'effect_mention_tw'},axis=1).reset_index()
# Precompute a mapping of event_id to mention_tw for faster lookups
event_id_to_mention = events_df.set_index('event_id')['mention_tw'].to_dict()

def get_pairs_fast(row):
    relation_to_mentions = []
    ids_pairs = []
    for k, v in row['causal_relations'].items():  # Iterate through key-value pairs
        for item in v:
            # Look up mentions directly from the precomputed dictionary
            mention1 = event_id_to_mention.get(item[0], None)
            mention2 = event_id_to_mention.get(item[1], None)

            # Append pair if both mentions exist
            if mention1 is not None and mention2 is not None:
                relation_to_mentions.append([mention1, mention2])
                ids_pairs.append([item[0], item[1]])
    return pd.Series({'mention_pairs': relation_to_mentions, 'ids_pairs': ids_pairs}) # Return a Series with named columns

# Apply the optimized function and store the result in a new column
# The result_type='expand' ensures that the returned Series is expanded into columns
processed_df = ev_df.apply(get_pairs_fast, axis=1, result_type='expand')
ev_df = pd.concat([ev_df, processed_df], axis=1)

In [458]:
ev_test_df = ev_test_df.join(valid_df.set_index('id')['causal_relations'])
causal_test_df = causal_test_df.set_index('cause_event').join(events_test_df.set_index('event_id').loc[causal_test_dft['cause_event']]['mention_tw'])
test_event_id_to_mention = events_test_df.set_index('event_id')['mention_tw'].to_dict()



In [466]:
def get_pairs_fast(row):
    relation_to_mentions = []
    ids_pairs = []
    for k, v in row['causal_relations'].items():  # Iterate through key-value pairs
        for item in v:
            # Look up mentions directly from the precomputed dictionary
            mention1 = event_id_to_mention.get(item[0], None)
            mention2 = event_id_to_mention.get(item[1], None)

            # Append pair if both mentions exist
            if mention1 is not None and mention2 is not None:
                relation_to_mentions.append([mention1, mention2])
                ids_pairs.append([item[0], item[1]])
    return pd.Series({'mention_pairs': relation_to_mentions, 'ids_pairs': ids_pairs}) # Return a Series with named columns

processed_test_df = ev_test_df.apply(get_pairs_fast, axis=1, result_type='expand')
ev_test_df = pd.concat([ev_test_df, processed_test_df], axis=1)

In [463]:
def process_row(row, tokenizer, max_length):
    stream = row['event_stream']  # list of events
    event_ids = row['event_stream_ids']  # list of event IDs
    mentions_pairs = row['mention_pairs']  # list of true cause-effect pairs
    ids_pairs = row['ids_pairs']  # list of true cause-effect pairs IDs
    rel_dict = {}

    bert_idx = 1  # account for CLS

    event_idx = 0
    words_to_encode = []
    for event in stream:
        event_id = event_ids[event_idx]
        event_tokens = tokenizer.tokenize(event)
        rel_dict[event_id] = {
            'event': event,
            'bert_start': bert_idx,
            'bert_end': bert_idx + len(event_tokens),
            'tokens': event_tokens
        }

        bert_idx += len(event_tokens)
        words_to_encode.append(event)
        event_idx += 1

    # Encode the original words
    encoding = tokenizer.encode_plus(
        words_to_encode,
        is_split_into_words=True,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_offsets_mapping=True,
        return_token_type_ids=False
    )
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # Extract BERT-encoded indices for true pairs
    true_pairs = []
    for cause, effect in ids_pairs:
        if cause in rel_dict and effect in rel_dict:
            cause_start = rel_dict[cause]['bert_start']

            cause_end = rel_dict[cause]['bert_end']

            effect_start = rel_dict[effect]['bert_start']

            effect_end = rel_dict[effect]['bert_end']

            true_pairs.append((cause_start, cause_end, effect_start, effect_end))

    return pd.Series({
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'true_pairs': true_pairs,
        'rel_dict': rel_dict
    })

def prepare_df(d, model_name="SpanBERT/spanbert-base-cased", max_length=512):
    df = d.copy()
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

    # Apply the process_row function to each row
    processed = df.apply(process_row, axis=1, result_type='expand', args=(tokenizer, max_length))
    df[['input_ids', 'attention_mask', 'true_pairs', 'rel_dict']] = processed

    return df

#prepared = prepare_df(ev_df)

In [467]:
prepared_test = prepare_df(ev_test_df)

In [ ]:
from torch.utils.data import Dataset, DataLoader


In [ ]:
#prepared.head(5)

In [ ]:
# class CausalEventDataset(Dataset):
#     def __init__(self, df, max_length=512):
#         self.df = df
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]

#         input_ids = torch.tensor(row['input_ids'], dtype=torch.long)
#         attention_mask = torch.tensor(row['attention_mask'], dtype=torch.long)
#         true_pairs = torch.tensor(row['true_pairs'], dtype=torch.long)

#         # # Create start and end labels
#         # start_labels = torch.zeros(self.max_length, dtype=torch.float)
#         # end_labels = torch.zeros(self.max_length, dtype=torch.float)

#         # for cause_start, cause_end, effect_start, effect_end in true_pairs:
#         #     start_labels[cause_start] = 1
#         #     end_labels[cause_end] = 1
#         #     start_labels[effect_start] = 1
#         #     end_labels[effect_end] = 1

#         # return {
#         #     'input_ids': input_ids,
#         #     'attention_mask': attention_mask,
#         #     'true_pairs': true_pairs,
#         #     'start_labels': start_labels,
#         #     'end_labels': end_labels
#         # }

In [ ]:
# from sklearn.model_selection import train_test_split




# train_df, val_df = train_test_split(
#     prepared,
#     test_size=0.2,  # 20% of the data for validation
#     random_state=42  # For reproducibility
# )





In [ ]:
# val_dataset =CausalEventDataset(val_df)
# train_dataset =CausalEventDataset(val_df)

In [ ]:
# val_dataset.df

# DAG Exploration

The adjacency matrix captures many-to-many relationships by considering every possible causal pair, but the distance matrix should help mitigate some of that sparsity.

## tree structures

In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path

In [ ]:
#CORRECT

class Tree(object):
    def __init__(self, event_id=None, event_label=None):
        self.parent = None
        self.num_children = 0
        self.children = list()
        self.event_id = event_id
        self.event_label = event_label
        self.idx = None

    def add_child(self, child):
        child.parent = self
        self.num_children += 1
        self.children.append(child)

    def size(self):
        if getattr(self, '_size', False):
            return self._size
        count = 1
        for i in range(self.num_children):
            count += self.children[i].size()
        self._size = count
        return self._size

    def height(self):
        if getattr(self, '_height', False):
            return self._height
        count = 0
        if self.num_children > 0:
            for i in range(self.num_children):
                child_height = self.children[i].height()
                if child_height > count:
                    count = child_height
            count += 1
        self._height = count
        return self._height

    def depth(self):
        if getattr(self, '_depth', False):
            return self._depth
        count = 0
        if self.parent:
            count += self.parent.depth()
            count += 1
        self._depth = count
        return self._depth

    def __iter__(self):
        yield self
        for c in self.children:
            for x in c:
                yield x

    @staticmethod


    @staticmethod
    def head_to_tree(event_stream_ids, event_stream, causal_relations):
        nodes = {event_id: Tree(event_id, event_label) for event_id, event_label in zip(event_stream_ids, event_stream)}
        super_root = Tree("SUPER_ROOT", "SUPER_ROOT")

        # Keep track of nodes that are effects and their causes
        effect_to_causes = {event_id: set() for event_id in event_stream_ids}

        # First pass: add all nodes to super_root
        for i, event_id in enumerate(event_stream_ids):
            nodes[event_id].idx = i + 1  # Shift indices to make room for super-root
            super_root.add_child(nodes[event_id])

        # Second pass: establish causal relationships
        for cause, effect in causal_relations:
            if cause in nodes and effect in nodes:
                effect_to_causes[effect].add(cause)

        # Third pass: build the tree structure
        for effect, causes in effect_to_causes.items():
            if causes:
                # Remove the effect from super_root's children
                if nodes[effect] in super_root.children:
                    super_root.children.remove(nodes[effect])
                    super_root.num_children -= 1

                # Add the effect as a child to all its causes
                for cause in causes:
                    nodes[cause].add_child(nodes[effect])

        return super_root
    #   return ret
    def tree_to_adj(event_len, tree, directed=True, self_loop=False):
      ret = np.zeros((event_len, event_len), dtype=np.float32)

      def dfs(node):
          for child in node.children:
              if node.idx is not None and child.idx is not None:
                  ret[node.idx - 1, child.idx - 1] = 1
                  dfs(child)

      # Process all children of the super root
      for child in tree.children:
          dfs(child)

      if not directed:
          ret = ret + ret.T

      if self_loop:
          np.fill_diagonal(ret, 1)

      return ret


    @staticmethod
    def inputs_to_tree_reps(event_stream_ids, causal_relations, directed=False, self_loop=False):
        tree = Tree.head_to_tree(event_stream_ids, causal_relations)
        adj = Tree.tree_to_adj(len(event_stream_ids), tree, directed=directed, self_loop=self_loop)
        adj = adj.reshape(1, len(event_stream_ids), len(event_stream_ids))
        return adj

    def adj_mat_to_dist_mat(adj_mat, directed=True, self_loop=False):
      dist_matrix = shortest_path(csgraph=csr_matrix(adj_mat), directed=directed)
      if self_loop:
          np.fill_diagonal(dist_matrix, 1)
      return dist_matrix


    def tree_to_dist(sent_len, tree):
      ret = -1 * np.ones(sent_len, dtype=np.int64)

      for node in tree:
          ret[node.idx] = node.dist

      return ret

    def visualize(self, level=0, visited=None):
      if visited is None:
          visited = set()

      indent = "  " * level
      print(f"{indent}{self.event_id} (Label: {self.event_label})")

      # Base case: Check if the node has already been visited
      if self.event_id in visited:
          return

      visited.add(self.event_id)

      for child in self.children:
          child.visualize(level + 1, visited)


In [ ]:
def get_multiple_relations(event, all_relations):
    """
    Helper function: Get all the causes and effects for a given event.
    """
    causes = [cause for cause, effect in all_relations if effect == event]
    effects = [effect for cause, effect in all_relations if cause == event]
    return causes, effects


In [ ]:
def get_multiple_relations(event, all_relations):
    """
    Helper function: Get all the causes and effects for a given event.
    """
    causes = [cause for cause, effect in all_relations if effect == event]
    effects = [effect for cause, effect in all_relations if cause == event]
    return causes, effects

class Sentence(object):
    def __init__(self, _id=None):
        self._id = _id
        self._words = []
        self._events = []
        self._event_ids = []
        self._causal_relations = []

    @property
    def event_ids(self) -> list:
        return self._event_ids

    @event_ids.setter
    def event_ids(self, param: list) -> None:
        assert isinstance(param, list)
        self._event_ids = param

    @property
    def id(self) -> str:
        return self._id

    @property
    def words(self) -> list:
        return self._words

    @words.setter
    def words(self, param: list) -> None:
        assert isinstance(param, list)
        self._words = param

    @property
    def events(self) -> list:
        return self._events

    @events.setter
    def events(self, param: list) -> None:
        assert isinstance(param, list)
        self._events = param

    @property
    def causal_relations(self) -> list:
        return self._causal_relations

    @causal_relations.setter
    def causal_relations(self, param: list) -> None:
        self._causal_relations = param



    def __len__(self):
        return len(self.words)

    def struct_position(self) -> tuple:
        head = [int(h) for h in self.head]
        nodes = [Tree() for _ in head]
        for i in range(len(nodes)):
            h = head[i]
            nodes[i].idx = i
            nodes[i].dist = -1  # just a filler
            if h == 0:
                root = nodes[i]
            else:
                nodes[h - 1].add_child(nodes[i])
        return root, [node.depth() for node in nodes]

    def adj_mat(self, directed=False, self_loop=False):
        if not hasattr(self, '_adj_mat'):
            self._adj_mat = Tree.inputs_to_tree_reps(self.event_ids, self.causal_relations, directed=directed, self_loop=self_loop)
        return self._adj_mat

    def dist_mat(self, directed=False, self_loop=False):
        if not hasattr(self, '_dist_mat'):
            adj_mat = self.adj_mat(directed, self_loop)
            self._dist_mat = Tree.adj_mat_to_dist_mat(adj_mat, directed=directed, self_loop=self_loop)
        return self._dist_mat




In [461]:
def get_multiple_relations(event, all_relations):
    """
    Helper function: Get all the causes and effects for a given event.
    """
    causes = [cause for cause, effect in all_relations if effect == event]
    effects = [effect for cause, effect in all_relations if cause == event]
    return causes, effects

class Sentence(object):
    def __init__(self, _id=None):
        self._id = _id
        self._words = []
        self._events = []
        self._event_ids = []
        self._causal_relations = []

    @property
    def event_ids(self) -> list:
        return self._event_ids

    @event_ids.setter
    def event_ids(self, param: list) -> None:
        assert isinstance(param, list)
        self._event_ids = param

    @property
    def id(self) -> str:
        return self._id

    @property
    def words(self) -> list:
        return self._words

    @words.setter
    def words(self, param: list) -> None:
        assert isinstance(param, list)
        self._words = param

    @property
    def events(self) -> list:
        return self._events

    @events.setter
    def events(self, param: list) -> None:
        assert isinstance(param, list)
        self._events = param

    @property
    def causal_relations(self) -> list:
        return self._causal_relations

    @causal_relations.setter
    def causal_relations(self, param: list) -> None:
        self._causal_relations = param



    def __len__(self):
        return len(self.words)

    def struct_position(self) -> tuple:
        head = [int(h) for h in self.head]
        nodes = [Tree() for _ in head]
        for i in range(len(nodes)):
            h = head[i]
            nodes[i].idx = i
            nodes[i].dist = -1  # just a filler
            if h == 0:
                root = nodes[i]
            else:
                nodes[h - 1].add_child(nodes[i])
        return root, [node.depth() for node in nodes]

    def adj_mat(self, directed=False, self_loop=False):
        if not hasattr(self, '_adj_mat'):
            self._adj_mat = Tree.inputs_to_tree_reps(self.event_ids, self.causal_relations, directed=directed, self_loop=self_loop)
        return self._adj_mat

    def dist_mat(self, directed=False, self_loop=False):
        if not hasattr(self, '_dist_mat'):
            adj_mat = self.adj_mat(directed, self_loop)
            self._dist_mat = Tree.adj_mat_to_dist_mat(adj_mat, directed=directed, self_loop=self_loop)
        return self._dist_mat



def remove_transitive_relations(event_pairs):
    # Convert event pairs (lists) to tuples before creating the set
    causal_relations = set(tuple(pair) for pair in event_pairs)
    transitive_relations_to_remove = set()

    # Identify and remove transitive relations
    for cause, effect in list(causal_relations):
        for intermediate in list(causal_relations):
            if intermediate[0] == effect and (cause != intermediate[1]):
                transitive_relations_to_remove.add((cause, intermediate[1]))

    # Remove transitive relations from the original set
    causal_relations = causal_relations - transitive_relations_to_remove

    return list(transitive_relations_to_remove), [list(pair) for pair in causal_relations]

def process_tree_row(row):
    # Get the event pairs for the row
    event_pairs = row['ids_pairs']

    # Apply remove_transitive_relations to get the transitive relations and cleaned causal relations
    transitive_relations, main_causal_relations = remove_transitive_relations(event_pairs)

    return pd.Series({
        'transitive_relations': transitive_relations,
        'main_causal_relations': main_causal_relations
    })


def prepare_tree_df(df):
    df = df.copy()

    processed = df.apply(process_tree_row, axis=1, result_type='expand')


    df[['transitive_relations', 'main_causal_relations']] = processed

    return df

#treed=prepare_tree_df(ev_df)

def process_dataframe(df):
    examples = []

    for idx, row in df.iterrows():
        sentence = Sentence(row.index)
        sentence.words = row['tokens']
        sentence.events = row['event_stream']
        sentence.event_ids = row['event_stream_ids']
        sentence.causal_relations = row['main_causal_relations']

        examples.append(sentence)

    return examples

In [ ]:
treed=prepare_tree_df(ev_df)

In [468]:
treed_test = prepare_tree_df(ev_test_df)

In [ ]:
def process_dataframe(df):
    examples = []

    for idx, row in df.iterrows():
        sentence = Sentence(row.index)
        sentence.words = row['tokens']
        sentence.events = row['event_stream']
        sentence.event_ids = row['event_stream_ids']
        sentence.causal_relations = row['main_causal_relations']

        examples.append(sentence)

    return examples

In [ ]:
treed.iloc[42]['main_causal_relations']

[['EVENT_e8fe45a5deaed25f49541f396c40be46',
  'EVENT_dd9f9367edfd76cba78043ba49600954'],
 ['EVENT_5351d0e1a065829b466f98eecb831889',
  'EVENT_3be7f7d184d4524ee10681dc73572a3e'],
 ['EVENT_098b3715292c95149d823b234f6ccef0',
  'EVENT_a1ad08f153d038e38c03fdbbf3f6150e'],
 ['EVENT_d881450d5e2645565fc0004d11cd8f00',
  'EVENT_2e8e37d6ff163c38105ba9e58f845f27'],
 ['EVENT_bb906c14a3d45785b1267a746eb84493',
  'EVENT_400ad1a1086302e02e4ac8f7f8d8f7dd'],
 ['EVENT_7d4ce77dbca3438c3aa6f68c09cff2c7',
  'EVENT_098b3715292c95149d823b234f6ccef0'],
 ['EVENT_25a3d586bfec5ce571619674721aac01',
  'EVENT_995af20ca70c5e3004593f483ffb0b8c'],
 ['EVENT_098b3715292c95149d823b234f6ccef0',
  'EVENT_25a3d586bfec5ce571619674721aac01'],
 ['EVENT_3be7f7d184d4524ee10681dc73572a3e',
  'EVENT_35b31ae7ab5714616e988c2084e61142'],
 ['EVENT_f83c395fb0eca21c05532e7089d6739c',
  'EVENT_400ad1a1086302e02e4ac8f7f8d8f7dd'],
 ['EVENT_a1ad08f153d038e38c03fdbbf3f6150e',
  'EVENT_dd9f9367edfd76cba78043ba49600954'],
 ['EVENT_2e8e37d6ff16

Tree Visualization:
SUPER_ROOT (Label: SUPER_ROOT)
  EVENT_498b03568fb2ac3b237e2a5bc101058a (Label: revolt)
  EVENT_e07ac4bc6d5e25ac441eeb8c60abcef6 (Label: took place)
  EVENT_070f93af55a14fafa910977b2371c773 (Label: removed)
    EVENT_9fc12da4e7e65bdb55f67077bdffc939 (Label: assumption)
      EVENT_ed4cb1af3702e5ba61024cc7ef534536 (Label: protest)
  EVENT_acf30935f45e33b40dd64a4acbcb58b5 (Label: events)
  EVENT_e2f64a7996915a23bf187ddb25ef1504 (Label: occurred)
    EVENT_2f3655195942b4649e07456d59895802 (Label: called)
  EVENT_7a67c3e0df294c0429627a36e4cf9b35 (Label: suppressed)
  EVENT_575c13014cea5d7dd0eb069a79c1d817 (Label: took place)
  EVENT_fcb8819ef2d0ac34945e76f71d19b208 (Label: event)
    EVENT_a624f5f5183ec5e855d76574888da17e (Label: renamed)
      EVENT_b9974d4fdbd3531053d92098a4e31760 (Label: changed)
  EVENT_3bfaf331ae9442d699d4e9c618bee817 (Label: reformation)
    EVENT_bd7b2f802a9a37a7f5d01198cb46f9e3 (Label: encouraged)
  EVENT_455a298a482078fabd7f345c841dfa23 (Label:

In [ ]:
treed.iloc[42]['sentences']

['The Glina massacres were killings of Serb peasants in the town of Glina in the Independent State of Croatia (NDH) that occurred between May and August 1941, during World War II.',
 'The first wave of massacres in the town began on 11 or 12 May 1941, when a band of Ustaše led by Mirko Puk murdered a group of Serb men and boys in a Serbian Orthodox church before setting it on fire.',
 'The following day, approximately 100 Serb males were murdered by the Ustaše in the nearby village of Prekopi.',
 'Estimates of the overall number of Serbs killed from 11–13 May range from 260 to 417.',
 'Further killings in Glina occurred between 30 July and 3 August of that same year, when 700–2,000 Serbs were massacred by a group of Ustaše led by Vjekoslav Luburić.',
 'Ljubo Jednak, the only survivor of these killings, went on to testify at the trials of the several prominent figures in the NDH after the war.',
 'Puk was captured by British forces in 1945 while attempting to flee to Austria and was ext

In [ ]:




sts = process_dataframe(treed)




example = sts[42]
root = Tree.head_to_tree(example.event_ids, example.events, example.causal_relations)
print("Tree Visualization:")
root.visualize()

Tree Visualization:
SUPER_ROOT (Label: SUPER_ROOT)
  EVENT_7d4ce77dbca3438c3aa6f68c09cff2c7 (Label: massacres)
    EVENT_098b3715292c95149d823b234f6ccef0 (Label: killed)
      EVENT_a1ad08f153d038e38c03fdbbf3f6150e (Label: erected)
        EVENT_dd9f9367edfd76cba78043ba49600954 (Label: removed)
      EVENT_25a3d586bfec5ce571619674721aac01 (Label: built)
        EVENT_995af20ca70c5e3004593f483ffb0b8c (Label: converted)
  EVENT_96e570c5bb3915d17643a50b1773a580 (Label: killings)
  EVENT_5fc91a093b0c5f95e72c18d42f9b944e (Label: massacres)
  EVENT_01b4385fdc05b3988318bb9dd8946f58 (Label: began)
  EVENT_bff6e23c61be0b72ae3bf78abf2d2769 (Label: murdered)
  EVENT_7dc40549c11fb7b2ca89e8d8014afc44 (Label: setting)
  EVENT_c1fec617e9777d7e73f1e576904447c2 (Label: on fire)
  EVENT_c08067c7514e361448720866c2a8534f (Label: murdered)
  EVENT_c6e0e23ae6f1445abb23c00fb155bb4b (Label: killed)
  EVENT_f83c395fb0eca21c05532e7089d6739c (Label: killings)
    EVENT_400ad1a1086302e02e4ac8f7f8d8f7dd (Label: te

In [ ]:
treed.iloc[7]['sentences']

['The Battle of Rocquencourt was a cavalry skirmish fought on 1 July 1815 in and around the villages of Rocquencourt and Le Chesnay.',
 'French dragoons supported by infantry and commanded by General Exelmans destroyed a Prussian brigade of hussars under the command of Lieutenant Colonel Eston von Sohr (who was severely wounded and taken prisoner during the skirmish).',
 'Prussian cavalry detachment under the command of Lieutenant Colonel Sohr ventured too far in advance of the main body of the Prussian army with the intention of reaching the Orléans road from Paris; where his detachment was to interrupt traffic on the road, and increase the confusion already produced in that quarter by the fugitives from the capital.',
 'However, when the Prussian detachment was in the vicinity of Rocquencourt it was ambushed by a superior French force.',
 'Under attack the Prussians retreated from Versailles and headed east, but were blocked by the French at Vélizy.',
 'They failed to re-enter Versai

In [ ]:




sts = process_dataframe(treed)




example = sts[7]
root = Tree.head_to_tree(example.event_ids, example.events, example.causal_relations)
print("Tree Visualization:")
root.visualize()

Tree Visualization:
SUPER_ROOT (Label: SUPER_ROOT)
  EVENT_ca8e7a084029b8b7e02eb8842acb141f (Label: Battle)
    EVENT_d2faf24fb4086f49cfef3e8014dd3781 (Label: supported)
      EVENT_2f9194041946343afae4076b4a914512 (Label: destroyed)
    EVENT_483a01318abb41f4245d664ff0bd490c (Label: commanded)
  EVENT_e65e2c3d3a42e514ad23446c912950bd (Label: command)
  EVENT_d8c5e375163e6235d40b4260da764ae5 (Label: wounded)
  EVENT_2d90e40c703c2cb9fa3ad3d147113ef9 (Label: taken)
  EVENT_8ec16546e047fec7e22aa769ae267012 (Label: command)
    EVENT_22c6e132281aa7754646b01cfae031af (Label: ventured)
      EVENT_101ac6c53adcbbeead2630740356836d (Label: ambushed)
        EVENT_b374c762a61a1b07126c71525d63a4c5 (Label: retreated)
          EVENT_f7dc4b4002a9bb1857f9c5943a7fa697 (Label: blocked)
            EVENT_7dcd4c35a18eafcd79bdd8f2ed8890d6 (Label: headed)
  EVENT_3de256733dc4c4c0db9fc61e158699b9 (Label: reaching)
    EVENT_b0f5453c636e130e5185f688140a1cc4 (Label: increase)
  EVENT_406a1f0aa7237ec3b9a9a65

In [ ]:
adj_mat =Tree.tree_to_adj(len(example.event_ids), root)

In [ ]:
adj_mat

array([[0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 

In [ ]:
dist_matrix = Tree.adj_mat_to_dist_mat(adj_mat)
dist_matrix

array([[ 0.,  1.,  1.,  2., inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       [inf,  0., inf,  1., inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf,  0., inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf,  0., inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf,  0., inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf,  0., inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf, inf,  0., inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, in

In [ ]:

def prepare_dist_df(df):
    df = df.copy()
    sts = process_dataframe(df)
    df.reset_index(inplace=True)
    df['dist_matrix'] = None
    for idx, row in df.iterrows():
        sentence = sts[idx]
        root = Tree.head_to_tree(sentence.event_ids, sentence.events, sentence.causal_relations)
        adj_mat = Tree.tree_to_adj(len(sentence.event_ids), root)
        dist_matrix = Tree.adj_mat_to_dist_mat(adj_mat)
        df.at[idx, 'dist_matrix'] = dist_matrix



    return df

dist_df=prepare_dist_df(treed)

## DAG structures

### naive dag (not ABCD-compatible)

In [ ]:
import gc

gc.collect()
#from graphical_models import DAG #jk

0

In [ ]:
class naiveDAG():
    def __init__(self):
        self.nodes = set()  # Set of nodes in the DAG
        self.edges = dict()  # Dictionary of edges: node -> set of child nodes
        self.event_labels = {}  # Dictionary to store event labels (event_id -> event_label)

    def add_node(self, node, event_label=None):
        """Add a node to the DAG, optionally with an event label (word)."""
        self.nodes.add(node)
        self.event_labels[node] = event_label
        if node not in self.edges:
            self.edges[node] = set()
        if event_label:
            if not hasattr(self, 'event_labels'):
                self.event_labels = {}
            self.event_labels[node] = event_label

    def add_edge(self, parent, child):
        """Add a directed edge from parent to child in the DAG."""
        if parent not in self.nodes:
            self.add_node(parent)
        if child not in self.nodes:
            self.add_node(child)
        self.edges[parent].add(child)

    def get_adj_mat(self):
        """Generate the adjacency matrix for the DAG."""
        node_list = list(self.nodes)
        node_idx_map = {node: idx for idx, node in enumerate(node_list)}
        adj_matrix = np.zeros((len(node_list), len(node_list)), dtype=np.float32)

        for parent, children in self.edges.items():
            parent_idx = node_idx_map[parent]
            for child in children:
                child_idx = node_idx_map[child]
                adj_matrix[parent_idx, child_idx] = 1

        return adj_matrix, node_idx_map

    def visualize(self):
        """Visualize the DAG, printing both event IDs and their labels (words) once, including disconnected nodes."""

        # Set to track disconnected nodes
        disconnected_nodes = set(self.nodes)

        # Print edges (parent -> child) for connected components
        for parent, children in self.edges.items():
            parent_label = self.event_labels.get(parent, "Unknown")
            for child in children:
                child_label = self.event_labels.get(child, "Unknown")
                print(f"{parent} ({parent_label}) -> {child} ({child_label})")
                disconnected_nodes.discard(child)
                disconnected_nodes.discard(parent)

        for node in disconnected_nodes:
                node_label = self.event_labels.get(node, "Unknown")
                print(f"Disconnected Node {node} ({node_label})")

# Function to directly build a DAG from event stream and causal relations
def create_dag_from_data(event_stream_ids, event_stream, causal_relations):
    dag = DAG()

    # First, add nodes for each event ID and their corresponding labels
    for event_id, event_label in zip(event_stream_ids, event_stream):
        dag.add_node(event_id, event_label)

    # Now, add edges from causal relations (cause -> effect)
    for cause, effect in causal_relations:
        dag.add_edge(cause, effect)

    return dag


In [ ]:
sts = process_dataframe(treed)

NameError: name 'process_dataframe' is not defined

In [ ]:
example = sts[7]
d = create_dag_from_data(example.event_ids, example.events, example.causal_relations)
d.visualize()

NameError: name 'sts' is not defined

In [ ]:
d.nodes

In [ ]:
d.edges

In [ ]:

def dag_df(df):
    df = df.copy()
    sts = process_dataframe(df)
    df.reset_index(inplace=True)
    df['edges'] = None
    df['nodes'] = None #just an alternatively formatted event stream bascially

    for idx, row in df.iterrows():
        sentence = sts[idx]
        d = create_dag_from_data(sentence.event_ids, sentence.events, sentence.causal_relations)
        df.at[idx, 'edges'] = d.edges
        df.at[idx, 'nodes'] = d.nodes



    return df

dag_df=dag_df(dist_df)

In [ ]:
set(dag_df.iloc[42]['event_stream_ids']) == dag_df.iloc[42]['nodes']

In [ ]:
#techincally this was for the causal apirs format
def get_pairs_fast(row):
    relation_to_mentions = []
    for pair in row['main_causal_relations']:  # Iterate through key-value pairs
            # Look up mentions directly from the precomputed dictionary
            mention1 = event_id_to_mention.get(pair[0], None)
            mention2 = event_id_to_mention.get(pair[1], None)

            # Append pair if both mentions exist
            if mention1 is not None and mention2 is not None:
                relation_to_mentions.append([mention1, mention2])
    return pd.Series({'main_causal_mentions': relation_to_mentions}) # Return a Series with named columns

mentions = dist_df.apply(get_pairs_fast, axis=1, result_type='expand')

### ABCD/ICD compatible DAG class

In [ ]:
!git clone https://github.com/IntelLabs/causality-lab.git
%cd causality-lab/

Cloning into 'causality-lab'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 248 (delta 18), reused 29 (delta 15), pack-reused 177 (from 1)
Receiving objects: 100% (248/248), 8.77 MiB | 32.89 MiB/s, done.
Resolving deltas: 100% (115/115), done.
/content/causality-lab/causality-lab


In [ ]:
from itertools import combinations
from graphical_models.basic_graph import Graph
from graphical_models.undirected_graph import UndirectedGraph
from graphical_models import arrow_head_types as Mark


In [ ]:
def get_multiple_relations(event, all_relations):
    """
    Helper function: Get all the causes and effects for a given event.
    """
    causes = [cause for cause, effect in all_relations if effect == event]
    effects = [effect for cause, effect in all_relations if cause == event]
    return causes, effects

class Sentence(object):
    def __init__(self, _id=None):
        self._id = _id
        self._words = []
        self._events = []
        self._event_ids = []
        self._causal_relations = []

    @property
    def event_ids(self) -> list:
        return self._event_ids

    @event_ids.setter
    def event_ids(self, param: list) -> None:
        assert isinstance(param, list)
        self._event_ids = param

    @property
    def id(self) -> str:
        return self._id

    @property
    def words(self) -> list:
        return self._words

    @words.setter
    def words(self, param: list) -> None:
        assert isinstance(param, list)
        self._words = param

    @property
    def events(self) -> list:
        return self._events

    @events.setter
    def events(self, param: list) -> None:
        assert isinstance(param, list)
        self._events = param

    @property
    def causal_relations(self) -> list:
        return self._causal_relations

    @causal_relations.setter
    def causal_relations(self, param: list) -> None:
        self._causal_relations = param



    def __len__(self):
        return len(self.words)

    def struct_position(self) -> tuple:
        head = [int(h) for h in self.head]
        nodes = [Tree() for _ in head]
        for i in range(len(nodes)):
            h = head[i]
            nodes[i].idx = i
            nodes[i].dist = -1  # just a filler
            if h == 0:
                root = nodes[i]
            else:
                nodes[h - 1].add_child(nodes[i])
        return root, [node.depth() for node in nodes]

    def adj_mat(self, directed=False, self_loop=False):
        if not hasattr(self, '_adj_mat'):
            self._adj_mat = Tree.inputs_to_tree_reps(self.event_ids, self.causal_relations, directed=directed, self_loop=self_loop)
        return self._adj_mat

    def dist_mat(self, directed=False, self_loop=False):
        if not hasattr(self, '_dist_mat'):
            adj_mat = self.adj_mat(directed, self_loop)
            self._dist_mat = Tree.adj_mat_to_dist_mat(adj_mat, directed=directed, self_loop=self_loop)
        return self._dist_mat



def remove_transitive_relations(event_pairs):
    # Convert event pairs (lists) to tuples before creating the set
    causal_relations = set(tuple(pair) for pair in event_pairs)
    transitive_relations_to_remove = set()

    # Identify and remove transitive relations
    for cause, effect in list(causal_relations):
        for intermediate in list(causal_relations):
            if intermediate[0] == effect and (cause != intermediate[1]):
                transitive_relations_to_remove.add((cause, intermediate[1]))

    # Remove transitive relations from the original set
    causal_relations = causal_relations - transitive_relations_to_remove

    return list(transitive_relations_to_remove), [list(pair) for pair in causal_relations]

def process_tree_row(row):
    # Get the event pairs for the row
    event_pairs = row['ids_pairs']

    # Apply remove_transitive_relations to get the transitive relations and cleaned causal relations
    transitive_relations, main_causal_relations = remove_transitive_relations(event_pairs)

    return pd.Series({
        'transitive_relations': transitive_relations,
        'main_causal_relations': main_causal_relations
    })


def prepare_tree_df(df):
    df = df.copy()

    processed = df.apply(process_tree_row, axis=1, result_type='expand')


    df[['transitive_relations', 'main_causal_relations']] = processed

    return df

treed=prepare_tree_df(ev_df)

def process_dataframe(df):
    examples = []

    for idx, row in df.iterrows():
        sentence = Sentence(row.index)
        sentence.words = row['tokens']
        sentence.events = row['event_stream']
        sentence.event_ids = row['event_stream_ids']
        sentence.causal_relations = row['main_causal_relations']

        examples.append(sentence)

    return examples

In [ ]:
class DAG(Graph):
    def __init__(self, nodes_set=None):
        super().__init__(nodes_set or set())
        self.edges = {node: set() for node in self.nodes_set}
        self.event_labels = {}
    def add_edge(self, parent, child):
        if parent not in self.nodes_set:
            self.add_node(parent)
        if child not in self.nodes_set:
            self.add_node(child)
        self.edges[parent].add(child)
        self._graph[parent].add(child)

    def remove_edge(self, parent, child):
        if (parent not in self.nodes_set) or (child not in self.nodes_set):
            raise ValueError("Node is not in the graph")
        self.edges[parent].discard(child)
        self._graph[parent].discard(child)

    def disconnect_node(self, node):
        if node not in self.nodes_set:
            raise ValueError("Node is not in the graph")
        # Remove outgoing edges
        self.edges[node].clear()
        self._graph[node].clear()
        # Remove incoming edges
        for parent in self.parents(node):
            self.edges[parent].discard(node)
            self._graph[parent].discard(node)

    # Override methods that need to maintain DAG properties
    def is_connected(self, node_i, node_j):
        return (node_j in self.edges[node_i]) or (node_i in self.edges[node_j])

    @property
    def nodes_set(self):
        return self._nodes_set

    @nodes_set.setter
    def nodes_set(self, value):
        self._nodes_set = value
        self.edges = {node: set() for node in self._nodes_set}
        self.create_empty_graph(self._nodes_set)

    def add_node(self, node, event_label=None):
        if node not in self.nodes_set:
            self.nodes_set.add(node)
            self.edges[node] = set()
            self._graph[node] = set()
        if event_label:
            self.event_labels[node] = event_label
        def create_complete_graph(self, nodes_set):
            self.nodes_set = set(nodes_set)
            self.edges = {node: set(nodes_set) - {node} for node in nodes_set}
            self._graph = self.edges.copy()


    def parents(self, node):
        return set(parent for parent, children in self.edges.items() if node in children)

    def find_children(self, node_parent, nodes_set=None):
        if nodes_set is None:
            nodes_set = self._nodes_set
        return set(child for child in nodes_set if child in self.edges.get(node_parent, set()))

    def get_adj_mat(self):
      node_list = list(self.nodes_set)
      node_idx_map = {node: idx for idx, node in enumerate(node_list)}
      adj_matrix = np.zeros((len(node_list), len(node_list)), dtype=np.float32)

      for parent, children in self.edges.items():
          parent_idx = node_idx_map[parent]
          for child in children:
              child_idx = node_idx_map[child]
              adj_matrix[parent_idx, child_idx] = 1

      return adj_matrix, node_idx_map

    def visualize(self):
        disconnected_nodes = set(self.nodes_set)
        for parent, children in self.edges.items():
            parent_label = self.event_labels.get(parent, "Unknown")
            for child in children:
                child_label = self.event_labels.get(child, "Unknown")
                print(f"{parent} ({parent_label}) -> {child} ({child_label})")
                disconnected_nodes.discard(child)
                disconnected_nodes.discard(parent)
        for node in disconnected_nodes:
            node_label = self.event_labels.get(node, "Unknown")
            print(f"Disconnected Node {node} ({node_label})")

    def find_children(self, node_parent, nodes_set=None):
        if nodes_set is None:
            nodes_set = self.nodes_set
        return set(child for child in nodes_set if child in self.edges.get(node_parent, set()))

    def get_ancestors(self, node, candidate_nodes=None):
        if candidate_nodes is None:
            candidate_nodes = self.nodes_set
        parents_set = self.parents(node) & candidate_nodes
        if len(parents_set) == 0:
            return {node}
        ancestors = set()
        for parent in parents_set:
            ancestors.update(self.get_ancestors(parent, candidate_nodes - parents_set))
        ancestors.add(node)
        return ancestors

    def get_skeleton_graph(self, en_nodes=None):
        if en_nodes is None:
            en_nodes = self.nodes_set

        skeleton = DAG(en_nodes.copy())
        for node_i, node_j in combinations(en_nodes, 2):
            if self.is_connected(node_i, node_j):
                skeleton.add_edge(node_i, node_j)
                skeleton.add_edge(node_j, node_i)  # Make it undirected

        return skeleton

    def is_ancestor(self, descendant_node, tested_node):
        if descendant_node == tested_node:
            return True
        parents_set = self.parents(descendant_node)
        if len(parents_set) == 0:
            return False
        if tested_node in parents_set:
            return True
        return any(self.is_ancestor(parent, tested_node) for parent in parents_set)

    def dsep(self, node_i, node_j, condition_set):
        ancestors = set()
        ancestors.update(self.get_ancestors(node_i))
        ancestors.update(self.get_ancestors(node_j))
        for cond_node in condition_set:
            ancestors.update(self.get_ancestors(cond_node))

        moral_graph = self.moralize(ancestors)
        return not moral_graph.is_reachable(node_i, node_j, condition_set)

    def get_ancestors(self, node, candidate_nodes=None):
        if candidate_nodes is None:
            candidate_nodes = self._nodes_set
        parents_set = self.parents(node) & candidate_nodes
        if len(parents_set) == 0:
            return {node}
        ancestors = set()
        for parent in parents_set:
            ancestors.update(self.get_ancestors(parent, candidate_nodes - parents_set))
        ancestors.add(node)
        return ancestors

    def moralize(self, ancestors):
        moral_graph = UndirectedGraph(ancestors)
        for node in ancestors:
            parents_set = self.parents(node) & ancestors
            for parent in parents_set:
                moral_graph.add_edge(parent, node)
            for parent_k, parent_l in combinations(parents_set, 2):
                if not self.is_connected(parent_k, parent_l):
                    moral_graph.add_edge(parent_k, parent_l)
        return moral_graph

    def is_connected(self, node_i, node_j):
        return (node_i in self.parents(node_j)) or (node_j in self.parents(node_i))

    def get_skeleton_graph(self, en_nodes=None):
        if en_nodes is None:
            en_nodes = self._nodes_set

        skeleton = DAG(en_nodes.copy())
        for node_i, node_j in combinations(en_nodes, 2):
            if self.is_connected(node_i, node_j):
                skeleton.add_edge(node_i, node_j)
                skeleton.add_edge(node_j, node_i)  # Make it undirected

        return skeleton

def create_dag_from_data(event_stream_ids, event_stream, causal_relations):
    dag = DAG()
    for event_id, event_label in zip(event_stream_ids, event_stream):
        dag.add_node(event_id, event_label)
    for cause, effect in causal_relations:
        dag.add_edge(cause, effect)
    return dag

In [470]:

def rawdag_df(df):
    df = df.copy()
    sts = process_dataframe(df)
    df.reset_index(inplace=True)
    df['edges'] = None
    df['nodes'] = None #just an alternatively formatted event stream bascially
    df['true_dag'] = None

    for idx, row in df.iterrows():
        sentence = sts[idx]
        d = create_dag_from_data(sentence.event_ids, sentence.events, sentence.causal_relations)
        df.at[idx, 'edges'] = d.edges
        df.at[idx, 'nodes'] = d._nodes_set
        df.at[idx, 'true_dag'] = d



    return df

rawdagdf=rawdag_df(treed)

In [471]:
rawdagdf_test=rawdag_df(treed_test)

# RAWDAG Modeling

## processing utils

In [ ]:
#auxmod = torch.load('/content/drive/MyDrive/W266/aux_model2.pth')

In [ ]:
event_id_to_mention = events_df.set_index('event_id')['mention_tw'].to_dict() #


### PAIN-based utils

In [ ]:
def get_pairs_fast(row):
    relation_to_mentions = []
    ids_pairs = []
    for item in row['main_causal_relations']:  # Iterate through key-value pairs
            # Look up mentions directly from the precomputed dictionary
            mention1 = event_id_to_mention.get(item[0], None)
            mention2 = event_id_to_mention.get(item[1], None)

            # Append pair if both mentions exist
            if mention1 is not None and mention2 is not None:
                relation_to_mentions.append([mention1, mention2])

    return pd.Series({'main_causal_mentions': relation_to_mentions}) # Return a Series with named columns

mentions = rawdagdf.apply(get_pairs_fast, axis=1, result_type='expand')
rawdagdf=rawdagdf.join(mentions)

In [473]:
def get_pairs_fast(row):
    relation_to_mentions = []
    ids_pairs = []
    for item in row['main_causal_relations']:  # Iterate through key-value pairs
            # Look up mentions directly from the precomputed dictionary
            mention1 = event_id_to_mention.get(item[0], None)
            mention2 = event_id_to_mention.get(item[1], None)

            # Append pair if both mentions exist
            if mention1 is not None and mention2 is not None:
                relation_to_mentions.append([mention1, mention2])

    return pd.Series({'main_causal_mentions': relation_to_mentions}) # Return a Series with named columns


In [474]:
mentions_test = rawdagdf_test.apply(get_pairs_fast, axis=1, result_type='expand')
rawdagdf_test=rawdagdf_test.join(mentions_test)

In [ ]:
len(rawdagdf.iloc[42]['main_causal_mentions'])

12

In [ ]:
def get_row_embeds(row, tokenizer, max_length=128):
    stream = row['event_stream']
    event_ids = row['event_stream_ids']
    ids_pairs = row['main_causal_relations']
    rel_dict = {}
    bert_idx = 0  # not using CLS

    words_to_encode = []
    for event_idx, (event, event_id) in enumerate(zip(stream, event_ids)):
        event_tokens = tokenizer.tokenize(event)
        rel_dict[event_id] = {
            'event': event,
            'bert_start': bert_idx,
            'bert_end': bert_idx + len(event_tokens),
            'tokens': event_tokens
        }
        bert_idx += len(event_tokens)
        words_to_encode.append(event)

    # Encode without padding
    encoding = tokenizer.encode_plus(
        words_to_encode,
        is_split_into_words=True,
        add_special_tokens=False,
        padding=False,
        truncation=False,
        return_attention_mask=True,
        return_token_type_ids=False
    )
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # Average tokens for each event

    avg_embeddings = []
    for event_id, event_info in rel_dict.items():
        start, end = event_info['bert_start'], event_info['bert_end']
        event_tokens = input_ids[start:end]
        avg_embedding = sum(event_tokens) // len(event_tokens)
        avg_embeddings.append(avg_embedding)

    # Manually pad to 128
    pad_token_id = tokenizer.pad_token_id
    padded_avg_embeddings = avg_embeddings[:max_length] + [pad_token_id] * max(0, max_length - len(avg_embeddings))

    new_rel_dict = {}
    current_position = 0
    for event_id, event_info in rel_dict.items():
        new_rel_dict[event_id] = {
            'event': event_info['event'],
            'bert_start': current_position,
            'bert_end': current_position + 1,
            'tokens': event_info['tokens']
        }
        current_position += 1
        if current_position >= max_length:
            break
    # Extract BERT-encoded indices for true pairs
    true_pairs = [
        (new_rel_dict[cause]['bert_start'], new_rel_dict[cause]['bert_end'],
         new_rel_dict[effect]['bert_start'], new_rel_dict[effect]['bert_end'])
        for cause, effect in ids_pairs
        if cause in new_rel_dict and effect in new_rel_dict
    ]
    new_attention_mask = [1] * len(avg_embeddings)
    new_attention_mask = new_attention_mask[:max_length] + [0] * max(0, max_length - len(new_attention_mask))

    return pd.Series({
        'input_ids': padded_avg_embeddings,
        'attention_mask': new_attention_mask,
        'true_pairs': true_pairs,
        'rel_dict': new_rel_dict
    })
def get_df_embeds(d, model_name="SpanBERT/spanbert-base-cased", max_length=128):
    df = d.copy()
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

    # Apply the process_row function to each row
    processed = df.apply(get_row_embeds, axis=1, result_type='expand', args=(tokenizer, max_length))
    df[['input_ids', 'attention_mask', 'true_pairs', 'rel_dict']] = processed

    return df



In [ ]:
embedded_df = get_df_embeds(rawdagdf)

In [475]:
embedded_df_test = get_df_embeds(rawdagdf_test)

In [ ]:
embedded_df.apply(lambda x: len(x['input_ids']), axis=1).sort_values(ascending=False)

,0
2912,128
2873,128
2874,128
2875,128
2876,128
...,...
4,128
5,128
6,128
7,128


In [ ]:
from torch.utils.data import Dataset, DataLoader


In [ ]:
from sklearn.model_selection import train_test_split


class DAGSet(Dataset):
    def __init__(self, df, max_length=128):
        self.df = df
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        input_ids = torch.tensor(row['input_ids'], dtype=torch.long)
        attention_mask = torch.tensor(row['attention_mask'], dtype=torch.long)
        event_stream_ids = row['event_stream_ids']
        true_dag = row['true_dag']
        rel_dict = row['rel_dict']

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'event_stream_ids': event_stream_ids,
            'true_dag': true_dag,
            'rel_dict': rel_dict

        }



In [ ]:
train_df, val_df = train_test_split(embedded_df, test_size=0.2, random_state=42)
train_df = DAGSet(train_df)
val_df = DAGSet(val_df)

In [476]:
test_df = DAGSet(embedded_df_test)

In [ ]:
import gc

In [ ]:
del processed_df, dup, events_df, full_df, causal_df, causal_dft, causal_data, mentions, rawdag_df
gc.collect()


24

In [ ]:
train_df.df.apply(lambda x: len(x['input_ids']), axis=1).sort_values(ascending=False)

,0
860,128
2351,128
1789,128
2366,128
930,128
...,...
1422,128
2605,128
461,128
1869,128


In [ ]:
gc.collect()

0

## modeling utils

In [ ]:
import sys
sys.path.append('.')
import random
import numpy as np
from causal_discovery_utils.constraint_based import LearnStructBase, unique_element_iterator
from graphical_models import PAG, PDSTree, arrow_head_types as Mark
from itertools import combinations, chain

from causal_discovery_utils.cond_indep_tests import CondIndepParCorr, DSep  # import a CI test that estimates partial correlation
from experiment_utils.synthetic_graphs import create_random_dag_with_latents, sample_data_from_dag
from causal_discovery_utils.performance_measures import calc_structural_accuracy_pag, score_family_dirichlet, score_bdeu
from causal_discovery_utils.data_utils import calc_stats
from causal_discovery_algs.icd import create_pds_tree
from causal_discovery_utils.cond_indep_tests import CondIndepParCorr


import torch
import torch.nn.functional as F



In [ ]:
class GPUPDSTree(PDSTree):
    def __init__(self, node_root, node_to_index, device='cuda'):
        super().__init__(node_root)
        self.device = device
        self.node_to_index = node_to_index

    def get_subsets_tensor(self, set_nodes, subset_size):
        with torch.no_grad():
            min_dist = torch.tensor([self.get_minimal_distance(node) for node in set_nodes], device=self.device)
            nodes_tensor = torch.tensor([self.node_to_index[node] for node in set_nodes], device=self.device)
            subsets = torch.combinations(nodes_tensor, r=subset_size)

            legal_subsets = []
            dist_sums = []

            for subset in subsets:
                original_subset = [list(self.node_to_index.keys())[list(self.node_to_index.values()).index(idx.item())] for idx in subset]
                if self.is_legal_cond_set(original_subset):
                    legal_subsets.append(subset)
                    dist_sums.append(min_dist[subset].sum())

            if legal_subsets:
                return torch.stack(legal_subsets), torch.stack(dist_sums)
            else:
                return torch.tensor([], device=self.device), torch.tensor([], device=self.device)

In [ ]:
def create_pds_tree(source_pag, node_root, node_to_index, en_nodes=None, max_depth=None, device='cuda'):
    with torch.no_grad():
        if en_nodes is not None:
            assert node_root in en_nodes
            assert en_nodes.issubset(source_pag.nodes_set)

        pds_tree = GPUPDSTree(node_root, node_to_index, device=device)

        adj_graph = source_pag.get_skeleton_graph(en_nodes=en_nodes)
        neighbors = adj_graph.get_neighbors(node_root)
        second_nodes = neighbors.copy()
        first_nodes = [node_root] * len(second_nodes)

        pds_nodes = neighbors.copy()
        for node_nb in neighbors:
            adj_graph.remove_edge(node_root, node_nb)

        if max_depth is None:
            max_depth = len(adj_graph.nodes_set) - 1

        first_nodes_trees = [pds_tree] * len(second_nodes)
        for node in pds_nodes:
            pds_tree.add_branch(node)
        second_nodes_trees = pds_tree.children.copy()

        while len(second_nodes) > 0:
            node_1 = first_nodes.pop(0)
            node_2 = second_nodes.pop(0)
            node_2_tree = second_nodes_trees.pop(0)

            if node_2_tree.depth_level >= max_depth:
                continue

            neighbors = adj_graph.get_neighbors(node_2)

            for node_3 in neighbors:
                if source_pag.is_possible_collider(node_x=node_1, node_middle=node_2, node_y=node_3):
                    adj_graph.remove_edge(node_2, node_3)
                    first_nodes.append(node_2)
                    second_nodes.append(node_3)
                    pds_nodes.append(node_3)

                    node_2_tree.add_branch(node_3)
                    added_branch = node_2_tree.get_child_branch(node_3)
                    second_nodes_trees.append(added_branch)
                    first_nodes_trees.append(node_2_tree)

        possible_d_sep_set = set(pds_nodes) - {node_root}
        return pds_tree, possible_d_sep_set

In [ ]:
# def score_bdeu(predicted_dag, true_dag, node_size = 2, en_nodes=None):
#     """
#     Calculate the BDeu score of a DAG
#     :param dag: DAG to be scored (icd output)
#     :param data: adjacency matrix generated by the true DAG
#     : node size: there are only 2 possible relationships ( --> or none)
#     :param en_nodes: the score will be calculated for the sub-graph induced by these nodes
#     :return: BDeu score
#     """
#     if dag is None:
#         return -float('inf')

#     if en_nodes is None:
#         en_nodes = dag.nodes_set

#     score = 0
#     for node in en_nodes:
#         parents = dag.parents(node)
#         family = tuple(parents) + (node, )  # a tuple family nodes where the child ("node") is last
#         family_sizes = [node_size[node_i] for node_i in family]
#         family_data = data[:, family]
#         counts = calc_stats(family_data, family_sizes)
#         if counts is None:  # handle memory error
#             return -float('inf')
#         counts = np.reshape(counts, [-1, family_sizes[-1]], order='F')  # 2nd axis is the states of the child

#         prior = np.ones_like(counts) / counts.size

#         score += score_family_dirichlet(counts=counts, prior=prior)

#     return score




In [ ]:
# class LearnStructICD(LearnStructBase):
#     def __init__(self, nodes_set, true_dag):
#         super().__init__(DAG, nodes_set=nodes_set)
#         self.graph.create_complete_graph(nodes_set)
#         self.edge_key = lambda x, y: (x, y) if x < y else (y, x)
#         self.conditioning_set = {self.edge_key(*edge): set() for edge in combinations(nodes_set, 2)}
#         self._state = dict(done=False, cond_set_size=0)
#         self.ci_test = DSep(true_dag, count_tests=True, use_cache=True)

#     def learn_structure(self):
#         max_cond_set_size = len(self.graph.nodes_set) - 2
#         while not self._state['done']:
#             self._state['done'] = self._learn_struct_incremental_step(self._state['cond_set_size'])
#             if self._state['cond_set_size'] > max_cond_set_size:
#               self._state['done'] = True
#             if self._state['done']:
#               break

#             self._state['cond_set_size'] += 1
#             print(f"Cond set size = {self._state['cond_set_size']}")
#             torch.cuda.empty_cache()
#             gc.collect()

#     def _learn_struct_incremental_step(self, cond_set_size):
#         print('Incremental Step')
#         cond_indep = self.ci_test.cond_indep
#         done = True

#         for node_i, node_j in combinations(self.graph.nodes_set, 2):
#             if not self.graph.is_connected(node_i, node_j):
#                 continue

#             cond_sets = self._get_pdsep_range_sets(node_i, node_j, cond_set_size)
#             for cond_set in cond_sets:
#                 print(f'Cond set: {cond_set}')
#                 done = False
#                 if self.ci_test.cond_indep(node_i, node_j, tuple(cond_set)):
#                     self.graph.delete_edge(node_i, node_j)
#                     self.sepset.set_sepset(node_i, node_j, tuple(cond_set))
#                     break

#             torch.cuda.empty_cache()
#             gc.collect()

#         if not done:
#             self.graph.orient_v_structures(self.sepset)
#             self.graph.maximally_orient_pattern(rules_set=[1, 2, 3, 4])
#             torch.cuda.empty_cache()
#             gc.collect()


#         return done

#     def _get_pdsep_range_sets(self, node_i, node_j, cond_set_size):
#         device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#         with torch.no_grad():
#             # Create pds_tree and possible d-separation sets on GPU
#             pds_tree_i, possible_d_sep_i = create_pds_tree(self.graph, node_i, max_depth=cond_set_size, device=device)
#             pds_tree_j, possible_d_sep_j = create_pds_tree(self.graph, node_j, max_depth=cond_set_size, device=device)

#             # Get subsets as tensors
#             cond_sets_tensor_i = pds_tree_i.get_subsets_tensor(set_nodes=possible_d_sep_i, subset_size=cond_set_size)
#             cond_sets_tensor_j = pds_tree_j.get_subsets_tensor(set_nodes=possible_d_sep_j, subset_size=cond_set_size)

#             # Combine and filter conditional sets directly on GPU
#             combined_sets = torch.cat([cond_sets_tensor_i, cond_sets_tensor_j], dim=0)
#             valid_sets = combined_sets[(combined_sets != node_i).all(dim=1) & (combined_sets != node_j).all(dim=1)]

#             # Return as list (convert at the end to minimize transfers)
#             result = valid_sets.cpu().tolist()

#         # Delete unnecessary tensors
#         del pds_tree_i, pds_tree_j, possible_d_sep_i, possible_d_sep_j
#         del cond_sets_tensor_i, cond_sets_tensor_j, combined_sets, valid_sets
#         torch.cuda.empty_cache()
#         gc.collect()

#         return result


In [ ]:
# def gumbel_softmax_sample(logits, tau=1.0):
#     """
#     Sample from Gumbel-Softmax distribution.
#     :param logits: logits of the categorical distribution (for edges in the DAG).
#     :param tau: temperature, controls the smoothness of the distribution.
#     :return: sample from Gumbel-Softmax distribution.
#     """
#     # Gumbel noise
#     gumbel_noise = -torch.log(-torch.log(torch.rand_like(logits)))

#     # Add Gumbel noise to logits
#     y = logits + gumbel_noise
#     return F.gumbel_softmax(y, tau=tau, hard=False)  # soft sample (continuous, differentiable)

# def learn_dag_with_gumbel_softmax(dag_logits, tau=1.0):
#     """
#     Learn the DAG structure by applying Gumbel-Softmax sampling.
#     :param dag_logits: logits representing edge probabilities in the DAG.
#     :param tau: temperature of the Gumbel-Softmax distribution.
#     :return: the learned DAG structure (probabilistic).
#     """
#     # Sample edges using Gumbel-Softmax
#     edge_probs = gumbel_softmax_sample(dag_logits, tau)

#     # Convert to a binary graph structure (optional, for visualization or further processing)
#     binary_dag = torch.round(edge_probs)  # Convert to 0 or 1 (hard selection)

#     return edge_probs, binary_dag

In [ ]:
from transformers import AutoModel

In [ ]:

class LearnStructBase:
    """
    Base class for constraint-based structure learning algorithms
    """
    def __init__(self, graph_class, nodes_set, **kwargs):
        if not isinstance(nodes_set, set):
            raise ValueError('nodes_set should be a set type')
        self.true_dag = kwargs.get('true_dag')
        self.sepset = SeparationSet(nodes_set)
        self.graph = graph_class(nodes_set)  # e.g., graph_class=PDAG (for CPDAG under causal sufficiency) or PAG
        self.graph.sepset = self.sepset  # link the algorithm's updated sepset to the graph (by reference)

    def get_graph(self):
        return self.graph.get_graph()


class SeparationSet:
    def __init__(self, nodes_set):
        self._sepset = {node: set() for node in nodes_set}
        self.nodes_set = nodes_set

    def erase(self):
        for node in self._sepset:
            self._sepset[node] = set()

    def set_sepset(self, node_1, node_2, sepset):
        self._sepset[node_1].add(node_2)
        self._sepset[node_2].add(node_1)
        self._sepset[node_1] |= set(sepset)
        self._sepset[node_2] |= set(sepset)

    def get_sepset(self, node_1, node_2):
        return self._sepset[node_1] & self._sepset[node_2]

    def copy(self, nodes=None, target_sepset=None):
        if nodes is None:
            nodes = self.nodes_set

        if target_sepset is None:
            target_sepset = SeparationSet(nodes)

        for node in nodes:
            target_sepset._sepset[node] = self._sepset[node].copy()

        return target_sepset

    def copy_from(self, source_sepset, nodes):
        for node in nodes:
            self._sepset[node] = source_sepset._sepset[node].copy()

def unique_element_iterator(chained_iterators):
    """
    return the unique instances of the chained iterators
    :param an iterator with possibly repeating elements, e.g., chained_iterators: chain(iter_a, iter_b)
    :return: an iterator with unique (unordered) elements
    """
    seen = set()
    for e in chained_iterators:
        if e in seen:
            continue

        seen.add(e)
        yield e

In [ ]:
train_df.df

,doc_id,title,sentences,tokens,event_stream,event_stream_labels,event_stream_ids,causal_relations,mention_pairs,ids_pairs,transitive_relations,main_causal_relations,edges,nodes,true_dag,main_causal_mentions,input_ids,attention_mask,true_pairs,rel_dict
2351,b94a8dea5aa40f195218d8461c18e47d,2009 ICC World Cricket League Division Three,[The 2009 ICC World Cricket League Division Th...,"[[The, 2009, ICC, World, Cricket, League, Divi...","[tournament, took place, formed, tournament, l...","[1, 1, 0, 0, 2, 2, 2, 2, 2, 0]","[EVENT_800cd336ec23d6c11627ec24be44a3b5, EVENT...",{'CAUSE': [['EVENT_15715069db5ce800ddcd260c54d...,"[[abandoned, replay], [tournament, reach], [to...","[[EVENT_15715069db5ce800ddcd260c54d3802d, EVEN...","[(EVENT_0daf94239d9319a2d8fcfa2692d02454, EVEN...","[[EVENT_0daf94239d9319a2d8fcfa2692d02454, EVEN...",{'EVENT_800cd336ec23d6c11627ec24be44a3b5': {'E...,"{EVENT_99d989a11636c699f9a0cb6ae7ca4d2b, EVENT...",<__main__.DAG object at 0x7960c8ccfa00>,"[[took place, cancellation], [tournament, lost...","[2348, 1271, 1824, 2348, 1575, 2519, 3928, 187...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[(1, 2, 8, 9), (0, 1, 4, 5), (0, 1, 5, 6), (1,...",{'EVENT_800cd336ec23d6c11627ec24be44a3b5': {'e...
1789,9d68b316accd49d8afbab7337f214a4b,Armagh rail disaster,[The Armagh rail disaster happened on 12 June ...,"[[The, Armagh, rail, disaster, happened, on, 1...","[disaster, negotiate, complete, stalled, decid...","[1, 0, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 0, ...","[EVENT_6260f0c11f492ce5232e1be96a13502e, EVENT...",{'CAUSE': [['EVENT_6260f0c11f492ce5232e1be96a1...,"[[disaster, killed], [colliding, injured], [co...","[[EVENT_6260f0c11f492ce5232e1be96a13502e, EVEN...","[(EVENT_d0601d1702f6f7e4a403c2035c144860, EVEN...","[[EVENT_d0601d1702f6f7e4a403c2035c144860, EVEN...",{'EVENT_6260f0c11f492ce5232e1be96a13502e': {'E...,"{EVENT_573587f68306921adfadaf69359a1476, EVENT...",<__main__.DAG object at 0x795f926f9000>,"[[divide, take], [injured, remains], [complete...","[7286, 14169, 2335, 22789, 1879, 13330, 1321, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[(5, 6, 6, 7), (12, 13, 13, 14), (2, 3, 3, 4),...",{'EVENT_6260f0c11f492ce5232e1be96a13502e': {'e...
2366,5fcee4f47ea942c4bbb800fa7421713c,Downing Street mortar attack,[The Downing Street mortar attack was carried ...,"[[The, Downing, Street, mortar, attack, was, c...","[attack, carried out, launched, assassinate, m...","[0, 1, 1, 0, 1, 2, 0, 2, 2, 2, 2, 2]","[EVENT_722363f64ccf564f34c591f145949bb3, EVENT...",{'CAUSE': [['EVENT_b4811745b30c1f46341b13ceb17...,"[[exploded, injuries], [exploded, hurt], [carr...","[[EVENT_b4811745b30c1f46341b13ceb17e6cae, EVEN...","[(EVENT_6b633b5bf436725318387d0f79d04e52, EVEN...","[[EVENT_6b633b5bf436725318387d0f79d04e52, EVEN...","{'EVENT_722363f64ccf564f34c591f145949bb3': {},...","{EVENT_1fc555f77dc8bffcd8c1f8b1f7f77448, EVENT...",<__main__.DAG object at 0x7960c8ccf3a0>,"[[launched, exploded], [exploded, injuries], [...","[2035, 1797, 2536, 8830, 2309, 6265, 1594, 973...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[(2, 3, 7, 8), (7, 8, 10, 11), (1, 2, 9, 10), ...",{'EVENT_722363f64ccf564f34c591f145949bb3': {'e...
930,20678659d988f0eff4ac59eb3399ab36,Elimination Chamber (2013),[Elimination Chamber (2013) (also known as No ...,"[[Elimination, Chamber, (, 2013, ), (, also, k...","[event, produced, took place, matches, contest...","[1, 0, 1, 0, 0, 1, 2, 2, 2, 2, 2, 2, 0, 0]","[EVENT_d37919ed73f448158efce980d57e5162, EVENT...",{'CAUSE': [['EVENT_c07388ffcb659e2c031cc368784...,"[[match, decide], [match, defeated], [took pla...","[[EVENT_c07388ffcb659e2c031cc36878444979, EVEN...",[],"[[EVENT_c07388ffcb659e2c031cc36878444979, EVEN...",{'EVENT_d37919ed73f448158efce980d57e5162': {'E...,"{EVENT_b937846a831571896ff7a38c065c2c11, EVENT...",<__main__.DAG object at 0x795f92435690>,"[[match, retaining], [match, defeated], [event...","[1856, 1666, 1271, 2697, 6839, 1801, 4958, 237...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ..."


        Initialize a CLEANN explainer [https://arxiv.org/abs/2310.20307].
        :param attention_matrix: self-attention, square matrix from which to infer inter-token relations.
        :param num_samples: number of samples from which the attention was calculated (e.g., config.hidden_size).
        :param p_val_th: p-value threshold for the partial-correlation-based independence test.
        :param explanation_tester: an externally-defined function that tests if a subset of tokens is an explanation.
        :param nodes_set: a list of tokens, where each token will be a node in the learned graph.
        :param search_minimal: If True, only the explanations with the minimal size will be returned.
        :param structure_learning_class: structure learning class to instantiate. Default: LearnStructICD.
        :param use_gumbel_softmax: Whether to use Gumbel-Softmax for continuous relaxation during structure learning.


In [339]:
class LearnStructICD(LearnStructBase):
    def __init__(self, nodes_set, node_to_index, true_dag, initial_graph, corr_matrix, corr_threshold=0.9):
        super().__init__(DAG, nodes_set=nodes_set)
        self.graph = initial_graph
        self.true_dag = true_dag
        self.corr_matrix = corr_matrix
        self.corr_threshold = corr_threshold
        self._state = dict(done=False, cond_set_size=0)
        self.node_to_index = node_to_index


    def learn_structure(self):
        max_cond_set_size = 5  # Maximum explaining set size
        while not self._state['done']:
            self._state['done'] = self._learn_struct_incremental_step(self._state['cond_set_size'])
            if self._state['cond_set_size'] >= max_cond_set_size:
                self._state['done'] = True
            if self._state['done']:
                break
            self._state['cond_set_size'] += 1
        return self.graph

    def _learn_struct_incremental_step(self, cond_set_size):
        done = True
        for node_i, node_j in combinations(self.graph.nodes_set, 2):
            if not self.graph.is_connected(node_i, node_j):
                continue
            i, j = self.node_to_index[node_i], self.node_to_index[node_j]


            if abs(self.corr_matrix[i, j]) <= self.corr_threshold:
                continue
            node_i_label = self.graph.event_labels.get(node_i, node_i)
            node_j_label = self.graph.event_labels.get(node_j, node_j)

            cond_sets = self._get_pdsep_range_sets(node_i, node_j, cond_set_size)
            for cond_set in cond_sets:
                if self.true_dag.dsep(node_i, node_j, tuple(cond_set)):
                    self.graph.delete_edge(node_i, node_j)
                    done = False
                    break
        if not done:
            self.graph.orient_v_structures()
            self.graph.maximally_orient_pattern(rules_set=[1, 2, 3, 4])
        return done

    def _get_pdsep_range_sets(self, node_i, node_j, cond_set_size):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        with torch.no_grad():
            pds_tree_i, possible_d_sep_i = create_pds_tree(self.graph, node_i, max_depth=cond_set_size, device=device, node_to_index=self.node_to_index)
            pds_tree_j, possible_d_sep_j = create_pds_tree(self.graph, node_j, max_depth=cond_set_size, device=device, node_to_index=self.node_to_index)

            possible_d_sep_i = [node for node in possible_d_sep_i if abs(self.corr_matrix[self.node_to_index[node_i], self.node_to_index[node]]) > self.corr_threshold]
            possible_d_sep_j = [node for node in possible_d_sep_j if abs(self.corr_matrix[self.node_to_index[node_j], self.node_to_index[node]]) > self.corr_threshold]

            cond_sets_tensor_i, _ = pds_tree_i.get_subsets_tensor(set_nodes=possible_d_sep_i, subset_size=cond_set_size)
            cond_sets_tensor_j, _ = pds_tree_j.get_subsets_tensor(set_nodes=possible_d_sep_j, subset_size=cond_set_size)

            if cond_sets_tensor_i.numel() > 0 and cond_sets_tensor_j.numel() > 0:
                combined_sets = torch.cat([cond_sets_tensor_i, cond_sets_tensor_j], dim=0)
                valid_sets = combined_sets[(combined_sets != node_i).all(dim=1) & (combined_sets != node_j).all(dim=1)]
                result = valid_sets.cpu().tolist()
            else:
                result = []

        return result

In [ ]:
def get_edge_order(corr_matrix):
    ''' prioritize stronger edges '''
    edges = []
    for i in range(corr_matrix.shape[0]):
        for j in range(i+1, corr_matrix.shape[1]):
            edges.append((i, j, abs(corr_matrix[i, j])))
    return sorted(edges, key=lambda x: x[2], reverse=True)




In [389]:
#reweighted attention with dag

class CLEANN:
    def __init__(self, attention_matrix, num_samples, true_dag, rel_dict, use_gumbel_softmax=False):
        self.attention_matrix = attention_matrix
        self.true_dag = true_dag
        self.rel_dict = rel_dict

        self.use_gumbel_softmax = use_gumbel_softmax
        self.corr_matrices, self.node_to_index = self.process_attention_matrices()
        self.graphs = self.initialize_graph_from_correlation(self.corr_matrices, threshold=0.9)

    def process_attention_matrices(self):
        attention_matrices = self.attention_matrix[-1]  # Use only the deepest layer
        corr_mats = []
        node_to_index_list = []
        batch_size = attention_matrices.size(0)
        with torch.no_grad():
          for batch_idx in range(batch_size):
              attention_matrix = attention_matrices[batch_idx]
              attention_matrix = torch.mean(attention_matrix, dim=0)


              num_vars = len(self.true_dag[batch_idx].nodes_set)
              corr_mat = torch.zeros((num_vars, num_vars), device=attention_matrix.device)
              nodes_list = list(self.true_dag[batch_idx].nodes_set)

              node_to_idx = {node: i for i, node in enumerate(self.true_dag[batch_idx].nodes_set)}


              event_attention = torch.zeros((num_vars, num_vars),
                                                    device=attention_matrix.device)

              for i, j in combinations(range(num_vars), 2):
                if i != j:
                  node_i = nodes_list[i]
                  node_j = nodes_list[j]
                  event_i_info = self.rel_dict[node_i]

                  event_j_info = self.rel_dict[node_j]


                  start_i = event_i_info['bert_start']
                  start_j = event_j_info['bert_start']
                  event_attention[i, j] = attention_matrix[start_i,start_j]


                      # Compute the full correlation matrix
              cov_matrix = torch.matmul(event_attention, event_attention.t())


              corr_mat = self.cov_to_corr(cov_matrix)
              corr_mats.append(corr_mat.cpu().numpy())
              node_to_index_list.append(node_to_idx)

          return corr_mats, node_to_index_list

    def initialize_graph_from_correlation(self, corr_mats, threshold):
        graphs = []
        for batch_idx, (corr_mat, node_to_index) in enumerate(zip(corr_mats, self.node_to_index)):
            nodes_set = set(node_to_index.keys())
            graph = DAG(nodes_set=nodes_set)

            for node_i, node_j in combinations(nodes_set, 2):
              i, j = node_to_index[node_i] - 1, node_to_index[node_j] - 1

              if abs(corr_mat[i, j]) > threshold:
                    graph.add_edge(node_i, node_j)

            graphs.append(graph)

        return graphs


    def cov_to_corr(self, cov_matrix):
        diag = torch.sqrt(torch.diag(cov_matrix) + 1e-8)  # Add small epsilon to avoid division by zero
        corr_matrix = cov_matrix / (torch.outer(diag, diag) + 1e-8)
        return torch.clamp(corr_matrix, -1.0, 1.0)

    def learn_graph(self):
        print('Initializing structure learning...')
        learned_graphs = []
        for batch_idx, (corr_matrix, graph, node_to_index) in enumerate(zip(self.corr_matrices, self.graphs, self.node_to_index)):
            true_dag = self.true_dag[batch_idx]

            icd_alg = LearnStructICD(nodes_set=graph.nodes_set,
                                    node_to_index=node_to_index,
                                    true_dag=true_dag,
                                    initial_graph=graph,
                                    corr_matrix=corr_matrix)

            print(f'Learning structure for batch item {batch_idx}...')
            icd_alg.learn_structure()
            print(f'Structure learned for batch item {batch_idx}.')

            if self.use_gumbel_softmax:
                learned_graphs.append(self.gumbel_softmax(graph))
            else:
                adj_matrix = graph.get_adj_mat()[0]
                edge_probs = torch.tensor(adj_matrix, dtype=torch.float32, requires_grad=True)

                learned_graphs.append(graph)

        self.graphs = learned_graphs
        return learned_graphs

    def gumbel_softmax(self, graph):
      adj_matrix = graph.get_adj_mat()[0]
      edge_probs = torch.tensor(adj_matrix, dtype=torch.float32, requires_grad=True)

      gumbel_softmax_output = F.gumbel_softmax(edge_probs, tau=1, hard=False) #returns logits!!


      return gumbel_softmax_output

    def align_graphs(self, icd_pag, true_dag):
        # Create node mapping
        node_mapping = {node: idx for idx, node in enumerate(true_dag.nodes_set)}

        # Create a new adjacency matrix for the ICD PAG
        n = len(true_dag.nodes_set)
        aligned_pag = np.zeros((n, n))


        # Reorder the ICD PAG so its correctly comparable
        for i, node_i in enumerate(icd_pag.nodes_set):
            for j, node_j in enumerate(icd_pag.nodes_set):
                if icd_pag.is_connected(node_i, node_j):
                    aligned_pag[node_mapping[node_i], node_mapping[node_j]] = 1

        # Get the true DAG adjacency matrix
        true_adj_mat = true_dag.get_adj_mat()[0]


        return aligned_pag, true_adj_mat

    def compare_with_true_dag(self):

        if self.graphs is None:
            self.graphs = self.learn_graph()
        losses = []
        for graph, true_dag in zip(self.graphs, self.true_dag):

            true_adj_mat = true_dag.get_adj_mat()[0]
            true_adj_mat_tensor = torch.tensor(true_adj_mat, dtype=torch.float32).clamp(0,1)
            bce_loss = F.binary_cross_entropy(graph, true_adj_mat_tensor)
            sparse_loss = bce_loss + .1*torch.sum(graph)
            losses.append(sparse_loss)
        return torch.mean(torch.stack(losses))
    def explain(self, target_node_idx, max_set_size=None, max_range=None):
        """
        Identify an explanation for the given target node. The result is stored in self.results['explanations'].
        :param target_node_idx: index of the node to be explained.
        :param max_set_size: setting a value limits the search to look for explanations
            having at most max_set_size nodes.
        :param max_range: setting a value limits the search to look for explanations
            such that the distance on the graph between the explanation nodes and the target node is at most max_range.
        :return: a list of minimal explanations (all explanations have the same size).
        """

        # learn a Graph if one haven't been learned already
        if self.graph is None:
            self.graph = self.learn_graph()

        # create a PDS-tree rooted at the target node
        pds_tree, full_explain_set = create_pds_tree(self.graph, target_node_idx, max_depth=max_range)
        max_pds_tree_depth = pds_tree.get_max_depth()
        results = dict()
        results['pds_tree'] = pds_tree
        results['full_explanation_set'] = full_explain_set
        results['max_pds_tree_depth'] = max_pds_tree_depth

        if max_set_size is None:
            max_size = len(full_explain_set)
        else:
            max_size = max_set_size

        explanations_list = []
        if self.is_explanation is None:
            if len(full_explain_set) <= max_size:
                explanations_list.append([full_explain_set, max_size])
        else:
            found_explanation = False
            for set_size in range(1, max_size+1):
                sets_list = pds_tree.get_subsets_tensor(set_nodes=full_explain_set, subset_size=set_size)
                sets_list.sort(key=lambda x: x[1])  # sort with respect to the sum of minimal distances
                for possible_explanation_set in sets_list:
                    if self.is_explanation(list(possible_explanation_set[0]), target_node_idx):
                        explanations_list.append(possible_explanation_set)
                        found_explanation = True
                if found_explanation and self._search_minimal:
                    break

        results['explanations'] = explanations_list
        self.results[target_node_idx] = results
        return explanations_list


In [384]:

class RAWDAG(nn.Module):
    def __init__(self, hidden_size=768, num_layers=4, num_heads=4, dropout=0.3, use_gumbel_softmax=False):
        super(RAWDAG, self).__init__()
        self.spanbert = AutoModel.from_pretrained("SpanBERT/spanbert-base-cased")
        self.adj_generator = nn.Linear(hidden_size, hidden_size)  # Generate adjacency matrix to learn

        self.encoder = PainfulEncoder(num_layers=num_layers,
                                         d_model=hidden_size,
                                         heads=num_heads,
                                         dropout=dropout)
        self.cleann = None

        self.oracle_dag = None


    def forward(self, input_ids, attention_mask):
        spanbert_output = self.spanbert(input_ids=input_ids, attention_mask=attention_mask)[0]
        adj_logits = self.adj_generator(spanbert_output)
        adj_mask = torch.sigmoid(adj_logits)

        encoder_output, attention_weights = self.encoder(spanbert_output, attention_mask, adj_mask=adj_mask)
        return encoder_output, attention_weights, adj_mask


    def explain(self, target_node_idx, max_set_size=None, max_range=None):
        if self.cleann is None:
            raise ValueError("CLEANN has not been initialized. Call initialize_cleann first.")
        return self.cleann.explain(target_node_idx, max_set_size, max_range)


    def initialize_oracle_dag(self, true_dag):
      self.oracle_dag = true_dag
      print('Initialized Oracle dag')

    def initialize_cleann(self, attention_matrix, num_samples, rel_dict):
        print('Initializing CLEANN...')
        self.cleann = CLEANN(attention_matrix, num_samples, true_dag=self.oracle_dag,
                              rel_dict=rel_dict)
        del attention_matrix
        torch.cuda.empty_cache()
        gc.collect()
        print('Initialized CLEANN.')

    def learn_dag(self):
        if self.cleann is None:
            raise ValueError("CLEANN has not been initialized. Call initialize_cleann first.")
        print('Learning graphs...')
        return self.cleann.learn_graph()

class PainfulEncoder(nn.Module):
    def __init__(self, num_layers, d_model, heads, dropout):
        super().__init__()
        self.layers = nn.ModuleList([
            PainfulEncoderLayer(d_model, heads, dropout)
            for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, mask, adj_mask=None):
        attention_weights = []
        for layer in self.layers:
            x, attn_weights = layer(x, mask, adj_mask)
            attention_weights.append(attn_weights.squeeze(1))
        return self.norm(x), attention_weights


class PainfulEncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout):
        super().__init__()
        self.self_attn = PainfulMultiHeadAttention(heads, d_model, dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask, adj_mask=None):
        attn_output, attn_weights = self.self_attn(x, x, x, mask, adj_mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x, attn_weights

class PainfulMultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout):
        super().__init__()
        self.d_k = d_model // heads
        self.h = heads
        self.linear_layers = nn.ModuleList([nn.Linear(d_model, d_model) for _ in range(3)])
        self.output_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None, adj_mask=None):
        batch_size = query.size(0)
        query, key, value = [l(x).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
                             for l, x in zip(self.linear_layers, (query, key, value))]
        scores = torch.matmul(query, key.transpose(-2, -1)) / (self.d_k ** 0.5)

        # Apply mask if available
        if mask is not None:
            mask = mask.unsqueeze(1).unsqueeze(2)  # (batch_size, 1, 1, seq_len)
            mask = mask.repeat(1, self.h, 1, 1)  # (batch_size, num_heads, 1, seq_len)
            scores = scores.masked_fill(mask == 0, -1e9)

        # Apply adjacency mask (Gumbel-Softmax or sigmoid)
        if adj_mask is not None:

            adj_mask = torch.sigmoid(adj_mask)

            seq_len = scores.shape[-1]  # Get the sequence length from the scores tensor
            adj_mask = adj_mask.unsqueeze(1).repeat(1, self.h, 1, 1)
            scores = scores * adj_mask[:, :, :, :seq_len]

        p_attn = F.softmax(scores, dim=-1)
        p_attn = self.dropout(p_attn)

        x = torch.matmul(p_attn, value)
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.h * self.d_k)

        return self.output_linear(x), p_attn


class PositionwiseFeedForward(nn.Module): #same as daddy longlegs
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Linear(d_model, d_model * 4)
        self.w_2 = nn.Linear(d_model * 4, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))





In [ ]:
import gc
gc.collect()

0

In [ ]:
def train_rawdag(model, train_dataloader, val_dataloader, num_epochs, learning_rate, device):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for i, batch in enumerate(train_dataloader):
            print(f'Train batch {i}')
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            event_ids = batch['event_stream_ids']
            true_dag = batch['true_dag']
            rel_dict = batch['rel_dict']

            _, attention_weights, _ = model(input_ids, attention_mask)

            model.initialize_oracle_dag(true_dag)
            del true_dag
            torch.cuda.empty_cache()
            gc.collect()


            model.initialize_cleann(attention_weights, len(event_ids), batch['rel_dict'])
            learned_dag = model.learn_dag()
            del attention_weights
            torch.cuda.empty_cache()
            gc.collect()

            loss = model.cleann.compare_with_true_dag()
            print(f'Loss: {loss}')
            loss.requires_grad_(True)


            loss.backward()
            optimizer.step()
            optimizer.zero_grad()


            total_train_loss += loss.item()



        avg_train_loss = total_train_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Average Training Loss: {avg_train_loss:.4f}")

        model.eval()
        total_val_loss = 0

        with torch.no_grad():
            for batch in val_dataloader:
                print('Beginning validation batch...')
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                event_ids = batch['event_stream_ids']
                true_dag = batch['true_dag']
                rel_dict = batch['rel_dict']

                _, attention_weights, _ = model(input_ids, attention_mask)
                model.initialize_oracle_dag(true_dag)
                del true_dag


                model.initialize_cleann(attention_weights, len(event_ids), batch['rel_dict'])
                del attention_weights
                torch.cuda.empty_cache()
                gc.collect()

                learned_dag = model.learn_dag()
                loss = model.cleann.compare_with_true_dag()
                print(f'Val loss: {loss}')

                total_val_loss += loss.item()

                del learned_dag
                torch.cuda.empty_cache()




        avg_val_loss = total_val_loss / len(val_dataloader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}")
        checkpoint_dir = '/content/drive/MyDrive/W266/checkpoints'

        os.makedirs(checkpoint_dir, exist_ok=True)
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch+1}.pt')
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': avg_train_loss,
            'val_loss': avg_val_loss,
        }, checkpoint_path)

        print(f"Checkpoint saved at: {checkpoint_path}")
        torch.cuda.empty_cache()

    return model

In [ ]:
def custom_collate(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    true_dag = [item['true_dag'] for item in batch]
    event_stream_ids = [item['event_stream_ids'] for item in batch]

    # Merge rel_dict items into a single dictionary
    merged_rel_dict = {}
    for item in batch:
        for event_id, event_info in item['rel_dict'].items():
            if event_id not in merged_rel_dict:
                merged_rel_dict[event_id] = event_info
            else:
                # If the event_id already exists, update the bert_start and bert_end
                existing_info = merged_rel_dict[event_id]
                existing_info['bert_start'] = min(existing_info['bert_start'], event_info['bert_start'])
                existing_info['bert_end'] = max(existing_info['bert_end'], event_info['bert_end'])
                existing_info['tokens'].extend(event_info['tokens'])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'event_stream_ids': event_stream_ids,
        'true_dag': true_dag,
        'rel_dict': merged_rel_dict
    }

In [ ]:
train_dataloader = DataLoader(train_df, batch_size=8, shuffle=True, collate_fn = custom_collate)
val_dataloader = DataLoader(val_df, batch_size=8, shuffle=False, collate_fn = custom_collate)

In [478]:
test_dataloader = DataLoader(test_df, batch_size=8, shuffle=True, collate_fn = custom_collate)

In [ ]:
from transformers import AutoModel

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mod = RAWDAG()
mod = mod.to(device)

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
gph= {'EVENT_99d989a11636c699f9a0cb6ae7ca4d2b': set(), 'EVENT_ada137a84bea66d1e077ecc047e270fb': {'EVENT_d6b8d998c7e72594a57b0a520cad78d1', 'EVENT_a8cf8fc39647ab5bdc57ffcc0e176ca0', 'EVENT_15715069db5ce800ddcd260c54d3802d', 'EVENT_0daf94239d9319a2d8fcfa2692d02454', 'EVENT_d9dd6a0fa3d0d138aeea6d82d32d0ddf', 'EVENT_800cd336ec23d6c11627ec24be44a3b5', 'EVENT_d549f030b764e16ced900edf0857f6af', 'EVENT_95f7e68b5e7aa8ee28c1ee6cfc06e664'}, 'EVENT_d6b8d998c7e72594a57b0a520cad78d1': {'EVENT_a8cf8fc39647ab5bdc57ffcc0e176ca0', 'EVENT_15715069db5ce800ddcd260c54d3802d', 'EVENT_0daf94239d9319a2d8fcfa2692d02454', 'EVENT_d9dd6a0fa3d0d138aeea6d82d32d0ddf', 'EVENT_800cd336ec23d6c11627ec24be44a3b5', 'EVENT_d549f030b764e16ced900edf0857f6af', 'EVENT_95f7e68b5e7aa8ee28c1ee6cfc06e664'}, 'EVENT_a8cf8fc39647ab5bdc57ffcc0e176ca0': {'EVENT_15715069db5ce800ddcd260c54d3802d', 'EVENT_0daf94239d9319a2d8fcfa2692d02454', 'EVENT_d9dd6a0fa3d0d138aeea6d82d32d0ddf', 'EVENT_800cd336ec23d6c11627ec24be44a3b5', 'EVENT_d549f030b764e16ced900edf0857f6af', 'EVENT_95f7e68b5e7aa8ee28c1ee6cfc06e664'}, 'EVENT_15715069db5ce800ddcd260c54d3802d': {'EVENT_0daf94239d9319a2d8fcfa2692d02454', 'EVENT_d9dd6a0fa3d0d138aeea6d82d32d0ddf', 'EVENT_800cd336ec23d6c11627ec24be44a3b5', 'EVENT_d549f030b764e16ced900edf0857f6af', 'EVENT_95f7e68b5e7aa8ee28c1ee6cfc06e664'}, 'EVENT_0daf94239d9319a2d8fcfa2692d02454': {'EVENT_800cd336ec23d6c11627ec24be44a3b5', 'EVENT_d549f030b764e16ced900edf0857f6af', 'EVENT_95f7e68b5e7aa8ee28c1ee6cfc06e664', 'EVENT_d9dd6a0fa3d0d138aeea6d82d32d0ddf'}, 'EVENT_d9dd6a0fa3d0d138aeea6d82d32d0ddf': {'EVENT_800cd336ec23d6c11627ec24be44a3b5', 'EVENT_d549f030b764e16ced900edf0857f6af', 'EVENT_95f7e68b5e7aa8ee28c1ee6cfc06e664'}, 'EVENT_800cd336ec23d6c11627ec24be44a3b5': {'EVENT_d549f030b764e16ced900edf0857f6af', 'EVENT_95f7e68b5e7aa8ee28c1ee6cfc06e664'}, 'EVENT_d549f030b764e16ced900edf0857f6af': {'EVENT_95f7e68b5e7aa8ee28c1ee6cfc06e664'}, 'EVENT_95f7e68b5e7aa8ee28c1ee6cfc06e664': set()}
tr =  {'EVENT_800cd336ec23d6c11627ec24be44a3b5': {'EVENT_a8cf8fc39647ab5bdc57ffcc0e176ca0', 'EVENT_15715069db5ce800ddcd260c54d3802d', 'EVENT_99d989a11636c699f9a0cb6ae7ca4d2b', 'EVENT_ada137a84bea66d1e077ecc047e270fb'}, 'EVENT_0daf94239d9319a2d8fcfa2692d02454': {'EVENT_a8cf8fc39647ab5bdc57ffcc0e176ca0', 'EVENT_15715069db5ce800ddcd260c54d3802d', 'EVENT_99d989a11636c699f9a0cb6ae7ca4d2b', 'EVENT_ada137a84bea66d1e077ecc047e270fb'}, 'EVENT_d6b8d998c7e72594a57b0a520cad78d1': set(), 'EVENT_d9dd6a0fa3d0d138aeea6d82d32d0ddf': set(), 'EVENT_ada137a84bea66d1e077ecc047e270fb': set(), 'EVENT_99d989a11636c699f9a0cb6ae7ca4d2b': set(), 'EVENT_15715069db5ce800ddcd260c54d3802d': {'EVENT_d549f030b764e16ced900edf0857f6af'}, 'EVENT_d549f030b764e16ced900edf0857f6af': set(), 'EVENT_a8cf8fc39647ab5bdc57ffcc0e176ca0': set(), 'EVENT_95f7e68b5e7aa8ee28c1ee6cfc06e664': set()}


In [ ]:
gph['EVENT_a8cf8fc39647ab5bdc57ffcc0e176ca0']

{'EVENT_0daf94239d9319a2d8fcfa2692d02454',
 'EVENT_15715069db5ce800ddcd260c54d3802d',
 'EVENT_800cd336ec23d6c11627ec24be44a3b5',
 'EVENT_95f7e68b5e7aa8ee28c1ee6cfc06e664',
 'EVENT_d549f030b764e16ced900edf0857f6af',
 'EVENT_d9dd6a0fa3d0d138aeea6d82d32d0ddf'}

In [ ]:
tr['EVENT_a8cf8fc39647ab5bdc57ffcc0e176ca0']

set()

In [ ]:
tr.keys()

dict_keys(['EVENT_800cd336ec23d6c11627ec24be44a3b5', 'EVENT_0daf94239d9319a2d8fcfa2692d02454', 'EVENT_d6b8d998c7e72594a57b0a520cad78d1', 'EVENT_d9dd6a0fa3d0d138aeea6d82d32d0ddf', 'EVENT_ada137a84bea66d1e077ecc047e270fb', 'EVENT_99d989a11636c699f9a0cb6ae7ca4d2b', 'EVENT_15715069db5ce800ddcd260c54d3802d', 'EVENT_d549f030b764e16ced900edf0857f6af', 'EVENT_a8cf8fc39647ab5bdc57ffcc0e176ca0', 'EVENT_95f7e68b5e7aa8ee28c1ee6cfc06e664'])

In [ ]:
gph.keys()

dict_keys(['EVENT_99d989a11636c699f9a0cb6ae7ca4d2b', 'EVENT_ada137a84bea66d1e077ecc047e270fb', 'EVENT_d6b8d998c7e72594a57b0a520cad78d1', 'EVENT_a8cf8fc39647ab5bdc57ffcc0e176ca0', 'EVENT_15715069db5ce800ddcd260c54d3802d', 'EVENT_0daf94239d9319a2d8fcfa2692d02454', 'EVENT_d9dd6a0fa3d0d138aeea6d82d32d0ddf', 'EVENT_800cd336ec23d6c11627ec24be44a3b5', 'EVENT_d549f030b764e16ced900edf0857f6af', 'EVENT_95f7e68b5e7aa8ee28c1ee6cfc06e664'])

In [ ]:
rdg = train_rawdag(mod, train_dataloader, val_dataloader, num_epochs=3, learning_rate=1e-3, device=device)

Streaming output truncated to the last 5000 lines.
Initializing CLEANN...
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
Learning structure for batch item 1...
Structure learned for batch item 1.
Learning structure for batch item 2...
Structure learned for batch item 2.
Learning structure for batch item 3...
Structure learned for batch item 3.
Learning structure for batch item 4...
Structure learned for batch item 4.
Learning structure for batch item 5...
Structure learned for batch item 5.
Learning structure for batch item 6...
Structure learned for batch item 6.
Learning structure for batch item 7...
Structure learned for batch item 7.
Loss: 0.2209300696849823
Train batch 148
Initialized Oracle dag
Initializing CLEANN...
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
Learning structu

RuntimeError: Parent directory checkpoints does not exist.

In [ ]:
rdg = train_rawdag(mod, train_dataloader, val_dataloader, num_epochs=3, learning_rate=1e-3, device=device)

Streaming output truncated to the last 5000 lines.
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
Learning structure for batch item 1...
Structure learned for batch item 1.
Learning structure for batch item 2...
Structure learned for batch item 2.
Learning structure for batch item 3...
Structure learned for batch item 3.
Learning structure for batch item 4...
Structure learned for batch item 4.
Learning structure for batch item 5...
Structure learned for batch item 5.
Learning structure for batch item 6...
Structure learned for batch item 6.
Learning structure for batch item 7...
Structure learned for batch item 7.
Loss: 0.20570871233940125
Train batch 148
Initialized Oracle dag
Initializing CLEANN...
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
Learning structure for batch item 1...

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mod2 = RAWDAG(num_layers=8)
mod2 = mod2.to(device)
rdg_tune = train_rawdag(mod2, train_dataloader, val_dataloader, num_epochs=3, learning_rate=1e-3, device=device)

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Streaming output truncated to the last 5000 lines.
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
Learning structure for batch item 1...
Structure learned for batch item 1.
Learning structure for batch item 2...
Structure learned for batch item 2.
Learning structure for batch item 3...
Structure learned for batch item 3.
Learning structure for batch item 4...
Structure learned for batch item 4.
Learning structure for batch item 5...
Structure learned for batch item 5.
Learning structure for batch item 6...
Structure learned for batch item 6.
Learning structure for batch item 7...
Structure learned for batch item 7.
Loss: 0.12273290753364563
Train batch 148
Initialized Oracle dag
Initializing CLEANN...
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
Learning structure for batch item 1...

In [346]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mod3 = RAWDAG(num_layers=4, use_gumbel_softmax=False)
mod3 = mod3.to(device)
rdg_reg = train_rawdag(mod3, train_dataloader, val_dataloader, num_epochs=5, learning_rate=1e-3, device=device)

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train batch 0
Initialized Oracle dag
Initializing CLEANN...
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
edge probs: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)
Learning structure for batch item 1...
Structure learned for batch item 1.
edge probs: tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
    

Exception ignored in: <function _xla_gc_callback at 0x796045316cb0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
edge probs: tensor([[0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], requires_grad=True)
Learning structure for batch item 1...
Structure learned for batch item 1.
edge probs: tensor([[0., 0., 0.,  ..., 0., 0., 

Exception ignored in: <function _xla_gc_callback at 0x796045316cb0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
edge probs: tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1., 1., 1., 0., 1., 0.],
        [0., 0., 0., 1., 1., 1., 1., 1., 1., 0.],
        [0., 0., 0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], requires_grad=True)
Learning structure for batch item 1...
Structure learned for batch item 1.
edge probs: tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
         0., 0., 0.],
        [0., 0., 0., 1., 

Exception ignored in: <function _xla_gc_callback at 0x796045316cb0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Initializing CLEANN...
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
edge probs: tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], requires_grad=True)
Learning structure for batch item 1...
Structure learned for batch item 1.
edge probs: tensor([[0.,

KeyboardInterrupt: 

In [320]:
rdg_tune.ev

RAWDAG(
  (spanbert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [380]:
def train_rawdag(model, train_dataloader, val_dataloader, num_epochs, learning_rate, device, sparsity_lambda=0.1):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for i, batch in enumerate(train_dataloader):
            print(f'Train batch {i}')
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            event_ids = batch['event_stream_ids']
            true_dag = batch['true_dag']
            rel_dict = batch['rel_dict']

            _, attention_weights, adj_mask = model(input_ids, attention_mask)

            model.initialize_oracle_dag(true_dag)
            del true_dag
            torch.cuda.empty_cache()
            gc.collect()

            model.initialize_cleann(attention_weights, len(event_ids), batch['rel_dict'])
            learned_dag = model.learn_dag()
            del attention_weights
            torch.cuda.empty_cache()
            gc.collect()

            loss = model.cleann.compare_with_true_dag()



            print(f'Loss: {loss.item()}')
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Average Training Loss: {avg_train_loss:.4f}")

        model.eval()
        total_val_loss = 0

        with torch.no_grad():
            for batch in val_dataloader:
                print('Beginning validation batch...')
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                event_ids = batch['event_stream_ids']
                true_dag = batch['true_dag']
                rel_dict = batch['rel_dict']

                _, attention_weights, adj_mask = model(input_ids, attention_mask)
                model.initialize_oracle_dag(true_dag)
                del true_dag

                model.initialize_cleann(attention_weights, len(event_ids), batch['rel_dict'])
                del attention_weights
                torch.cuda.empty_cache()
                gc.collect()

                learned_dag = model.learn_dag()
                loss = model.cleann.compare_with_true_dag()


                print(f'Val loss: {loss.item()}')

                total_val_loss += loss.item()

                del learned_dag
                torch.cuda.empty_cache()

        avg_val_loss = total_val_loss / len(val_dataloader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}")
        checkpoint_dir = '/content/drive/MyDrive/W266/checkpoints'

        os.makedirs(checkpoint_dir, exist_ok=True)
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpointB_epoch_{epoch+1}.pt')
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': avg_train_loss,
            'val_loss': avg_val_loss,
        }, checkpoint_path)

        print(f"Checkpoint saved at: {checkpoint_path}")
        torch.cuda.empty_cache()

    return model

In [381]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mod3 = RAWDAG(num_layers=4, use_gumbel_softmax=True)
mod3 = mod3.to(device)

rdg_reg = train_rawdag(mod3, train_dataloader, val_dataloader, num_epochs=5, learning_rate=1e-3, device=device)

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Streaming output truncated to the last 5000 lines.
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
Learning structure for batch item 1...
Structure learned for batch item 1.
Learning structure for batch item 2...
Structure learned for batch item 2.
Learning structure for batch item 3...
Structure learned for batch item 3.
Learning structure for batch item 4...
Structure learned for batch item 4.
Learning structure for batch item 5...
Structure learned for batch item 5.
Learning structure for batch item 6...
Structure learned for batch item 6.
Learning structure for batch item 7...
Structure learned for batch item 7.
Loss: 1.7574477195739746
Train batch 148
Initialized Oracle dag
Initializing CLEANN...
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
Learning structure for batch item 1...


## diagnostics: SHD

In [328]:
def structural_hamming_distance_dag(predicted_dag: DAG, true_dag: DAG):
    """
    Measure structural hamming distance between two DAGs for causal dependency evaluation.
    The following are calculated:
        * Edges (directed only)
            ** Missing: an edge missing from the predicted DAG but existing in the true DAG
            ** Extra:   an edge in the predicted DAG but missing from the true DAG
            ** Reversed: directed in opposite direction

    The total SHD is the sum of all values in the returned dictionary.

    :param predicted_dag: DAG predicted by the model
    :param true_dag: Ground truth DAG
    :return: A dictionary with SHD counts
    """
    shd_edge = {'missing': 0, 'extra': 0, 'reversed': 0}

    for parent in true_dag.nodes_set:
        true_children = true_dag.find_children(parent)
        predicted_children = predicted_dag.find_children(parent)

        # Missing edges: in true DAG but not in predicted DAG
        shd_edge['missing'] += len(true_children - predicted_children)

        # Extra edges: in predicted DAG but not in true DAG
        shd_edge['extra'] += len(predicted_children - true_children)

        # Reversed edges: check direction mismatches
        for child in true_children & predicted_children:
            if not predicted_dag.is_ancestor(child, parent):  # if reversed
                shd_edge['reversed'] += 1

    shd_total = sum(shd_edge.values())
    return {'total': shd_total, 'edge': shd_edge}


In [390]:
def evaluate_dag_predictions(model, val_dataloader, device):
    model.eval()
    total_shd = 0
    total_samples = 0
    shd_details = {'missing': 0, 'extra': 0, 'reversed': 0}

    with torch.no_grad():
        for batch in val_dataloader:
              input_ids = batch['input_ids'].to(device)
              attention_mask = batch['attention_mask'].to(device)
              event_ids = batch['event_stream_ids']
              true_dag = batch['true_dag']
              rel_dict = batch['rel_dict']

              _, attention_weights, _ = model(input_ids, attention_mask)
              model.initialize_oracle_dag(true_dag)



              model.initialize_cleann(attention_weights, len(event_ids), batch['rel_dict'])
              del attention_weights
              torch.cuda.empty_cache()
              gc.collect()

              learned_dag = model.learn_dag()

            # Compare predicted DAGs with true DAGs
              for pred_dag, true_dag in zip(learned_dag, true_dag):
                  shd_result = structural_hamming_distance_dag(pred_dag, true_dag)
                  total_shd += shd_result['total']
                  for key in shd_details:
                      shd_details[key] += shd_result['edge'][key]
                  total_samples += 1

    avg_shd = total_shd / total_samples
    avg_shd_details = {k: v / total_samples for k, v in shd_details.items()}

    return avg_shd, avg_shd_details

In [391]:
shd2, shd2_details = evaluate_dag_predictions(rdg_reg, val_dataloader, device)

Initialized Oracle dag
Initializing CLEANN...
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
Learning structure for batch item 1...
Structure learned for batch item 1.
Learning structure for batch item 2...
Structure learned for batch item 2.
Learning structure for batch item 3...
Structure learned for batch item 3.
Learning structure for batch item 4...
Structure learned for batch item 4.
Learning structure for batch item 5...
Structure learned for batch item 5.
Learning structure for batch item 6...
Structure learned for batch item 6.
Learning structure for batch item 7...
Structure learned for batch item 7.
Initialized Oracle dag
Initializing CLEANN...
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
Learning structure for batch item 1...
Structure learned for batch item 1.
Learning s

In [397]:
shd2

65.03602058319039

In [335]:
shd, shd_details = evaluate_dag_predictions(rdg_tune, val_dataloader, device)

Initialized Oracle dag
Initializing CLEANN...
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
Learning structure for batch item 1...
Structure learned for batch item 1.
Learning structure for batch item 2...
Structure learned for batch item 2.
Learning structure for batch item 3...
Structure learned for batch item 3.
Learning structure for batch item 4...
Structure learned for batch item 4.
Learning structure for batch item 5...
Structure learned for batch item 5.
Learning structure for batch item 6...
Structure learned for batch item 6.
Learning structure for batch item 7...
Structure learned for batch item 7.
Initialized Oracle dag
Initializing CLEANN...
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
Learning structure for batch item 1...
Structure learned for batch item 1.
Learning s

In [479]:
shd_test, shd_test_details = evaluate_dag_predictions(rdg_tune, test_dataloader, device)

Initialized Oracle dag
Initializing CLEANN...
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
Learning structure for batch item 1...
Structure learned for batch item 1.
Learning structure for batch item 2...
Structure learned for batch item 2.
Learning structure for batch item 3...
Structure learned for batch item 3.
Learning structure for batch item 4...
Structure learned for batch item 4.
Learning structure for batch item 5...
Structure learned for batch item 5.
Learning structure for batch item 6...
Structure learned for batch item 6.
Learning structure for batch item 7...
Structure learned for batch item 7.
Initialized Oracle dag
Initializing CLEANN...
Initialized CLEANN.
Learning graphs...
Initializing structure learning...
Learning structure for batch item 0...
Structure learned for batch item 0.
Learning structure for batch item 1...
Structure learned for batch item 1.
Learning s

In [337]:
shd_details

{'missing': 5.941680960548885, 'extra': 255.10634648370498, 'reversed': 0.0}

In [481]:
shd_test_details

{'missing': 0.0, 'extra': 58.595774647887325, 'reversed': 0.0}

## clear

In [ ]:
import gc

In [ ]:
del mod

In [ ]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            83Gi       1.1Gi        78Gi       190Mi       3.9Gi        81Gi
Swap:             0B          0B          0B
